In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

from utils_alertas import *

EJEMPLO:
202104 mes cerrado de la cosecha de 202104 [202011, 202012, 202101, 202102, 202103, 202104] [202102, 202103, 202104] 
 historia 
 [202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012, 202101, 202102, 202103, 202104] 
 ---------------------------------------------------------------------------------------------------- 
 [202105, 202106] 
 ---------------------------------------------------------------------------------------------------- 
 [202005, 202006]


In [2]:
filtro_banca_gran

<function utils_alertas.filtro_banca_gran(df, col_funnel='FUNNEL_4', cosechas_a_analizar=['202009', '202010', '202011', '202012', '202101', '202102'])>

In [3]:
#nativos
import os
import unicodedata

#terceros
import pandas as pd
#import pymssql
#import missingno as msno
from collections import Counter
import scorecardpy as sc

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

In [4]:
data = pd.read_csv('AVANCE_5.5.csv', dtype={
    'COD_UNICO': str,
    'PERIODO': str
})
data.head()

,PERIODO,COD_UNICO,RUC,FLG_VIGENTE,NOMBRE_COMPLETO,FLG_SUSPENDIDO,FLG_VENCIDA,FLG_CANCELADO,FLG_MONOLINEA,FLG_SOLO_REACTIVA,...,target_fusion_atraso_feve,PERIODO_ZONAL,target_FINAL_fusion_atraso_feve_2Mbueno,target_FINAL_fusion_atraso_SEGUIDO_15000_FEVE_2Mbueno,target_fusion_atraso_feve_3_3Mbueno,target_fusion_atraso_feve_3_1Mbueno,target_fusion_atraso_feve_3,target_variante_atraso_feve_3_3Mbueno,target_variante_atraso_feve_3_1Mbueno,target_variante_atraso_feve
0,202008,0002225225,20170717261,1,RUMI IMPORT,0,0,0,0,0,...,0.0,202008_BEL ZONAL 3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,202008,0004039624,10010600338,0,REATEGUI FLORES LUCAS,0,1,0,0,0,...,1.0,202008_BEP ZONAL 2,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN
2,202008,0004017117,20293331066,1,PRECISION PERU,1,1,1,0,0,...,NaN,202008_BEL ZONAL 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,202008,0003980042,20101391397,0,COMERCIAL INDUSTRIAL DELTA,1,0,1,0,1,...,NaN,202008_BEL ZONAL 1,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN
4,202008,0003290442,20100244129,1,AGRICOLA COMERCIAL E INDUSTRIAL,1,0,0,0,1,...,0.0,202008_BEL ZONAL 2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data.shape

(24014, 2659)

In [6]:
#data['CAMBIO_ACCIONISTAS_ANNIO'].value_counts(dropna=False)

In [7]:
print(len(data.columns))

2659


In [8]:
data['TIENE_REACTIVA_1_2_CON_OTRAS_EF'].value_counts()

0    16503
1     7511
Name: TIENE_REACTIVA_1_2_CON_OTRAS_EF, dtype: int64

In [9]:
#TIENE_DEBT_MTOSALDO

In [10]:
def oother_logic(ibk, rcc):
     #       ibk  rcc
    if pd.isnull(ibk):
         # ---> none none ---> none
         # ---> none 0 ---> none
         # ---> none 1 ---> none
        return None   
    elif ibk == 1:
        # ---> 1 none ---> 1
        # ---> 1 0 ---> 1
        # ---> 1 1 ---> 1
        return 1
    elif ibk == 0 and pd.isnull(rcc):
         # ---> 0 none ---> None     --------> ¿Hará tanta diferencia?
        return None
    elif ibk == 0 and rcc == 0:
          # ---> 0 0 ---> 0
        return 0
    elif ibk == 0 and rcc == 1:
         # ---> 0 none ---> 1       ------  
        return 1
    else:
        print(".")
        return None

data['target_SF_8_dias_atraso_o_mas_NEW'] = data[['target_8_dias_atraso_ibk', 'target_rcc_8_dias_atraso']].apply(
    lambda _: oother_logic(_[0], _[1]), axis=1
)
data['target_SF_8_dias_atraso_o_mas_NEW'].value_counts(dropna=False)

0.0    16667
NaN     6694
1.0      653
Name: target_SF_8_dias_atraso_o_mas_NEW, dtype: int64

In [11]:
data['target_fusion_atraso_feve_2'] = data[['target_SF_8_dias_atraso_o_mas_NEW', 'target_seguircovid_reducir_salir', 'target_feve_reducir_salir']].apply(
    lambda _: None if pd.isnull(_[0]) or pd.isnull(_[1]) or pd.isnull(_[2]) else (1 if _[0] == 1 or _[1] == 1 else 0) ,axis= 1
)
    
data['target_fusion_atraso_feve_2'].value_counts(dropna=False)

0.0    15123
NaN     8199
1.0      692
Name: target_fusion_atraso_feve_2, dtype: int64

In [12]:
col_target = 'target_FINAL_fusion_atraso_feve_2Mbueno'

#### OTROS BANCOS

In [13]:
[col for col in data.columns if 'BCP' in col.upper()]

['SOW_BCP_ULT_RCC',
 'MTOSALDO_BCP_V',
 'MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x',
 'PERCETN_DEBT_IN__BCPSF_ULT_RCC',
 'DEBT_IN__BCPSF_ULT_RCC',
 'MAX_DIAS_ATRASO_BCPSF_ULT_RCC_y',
 'TIENE_DEBT_MTOSALDO_BCP_SF_U3M',
 'DEBT_CAIDA_SEGUIDA_MTOSALDO_BCP_U3M_SF',
 'DEBT_SUBIDA_SEGUIDA_MTOSALDO_BCP_U3M_SF',
 'DEBT_MENOR_MTOSALDO_BCP_DE_U3M_SF',
 'DEBT_MAYOR_MTOSALDO_BCP_DE_U3M_SF',
 'DEBT_NO_MTOSALDO_BCP_SF_ESTE_MES',
 'DEBT_NO_MTOSALDO_BCP_SF_U2M',
 'DEBT_NO_MTOSALDO_BCP_SF_U3M',
 'DEBT_CAIDA_20PERCENT_MTOSALDO_BCP_SF_U3M',
 'DEBT_CAIDA_30PERCENT_MTOSALDO_BCP_SF_U3M',
 'DEBT_CAIDA_10PERCENT_MTOSALDO_BCP_SF_U3M',
 'DEBT_SUBIDA_20PERCENT_MTOSALDO_BCP_SF_U3M',
 'DEBT_SUBIDA_30PERCENT_MTOSALDO_BCP_SF_U3M',
 'DEBT_SUBIDA_10PERCENT_MTOSALDO_BCP_SF_U3M',
 'GAR_TOT_BCP',
 'GAR_PRENDA_BCP',
 'PERCENT_GAR_BCP',
 'GAR_AUTO_HIPO_PRENDA_BCP',
 'TIENE_MAYOR_GARTOTAL_EN_BCP',
 'GAR_HIPO_BCP',
 'PERCENT_GAR_AUTO_HIPO_BCP',
 'GAR_AUTO_BCP',
 'PERCENT_GAR_PREFERIDAS_BCP',
 'GAR_AUTO_HIPO_BCP',
 'MTO_DEUDA_FULL_RCC_B

## DATA A ANALIZAR

In [14]:
col_no_review = list(set(['COD_UNICO', 'PERIODO', 'DES_GRUPO_ECONOMICO', 'ENCARGADO', 
                 'NOMBRE_COMPLETO', 'NOMBRE_JEFE', 'RUC', 'GRUPO_ECON_FEVE_SEGUIR', 'MTO_CASH_IN', 'DESC_ATRASO_SF',
                 'GRUPO_ECON_CLASIF_NO_NORMAL', 'GRUPO_ECON_FEVE_DURO', 'RFMClass_septil', 'MTO_CASH_IN', 'DESC_ATRASO_SF',
                 'SUBSECTOR'] + [col for col in data.columns if 'DESCR' in col.upper()]))
col_no_review = list(set(col_no_review))
len(col_no_review)

20

In [15]:
all_data = data.copy()

In [16]:

print(data.shape)
data = filtro_banca_gran(data)
print(data.shape)

(24014, 2661)
(2488, 2661)


In [17]:
data['PERIODO'].value_counts(dropna=False)

202009    418
202010    417
202012    416
202011    415
202102    412
202101    410
Name: PERIODO, dtype: int64

In [18]:
data['ZONAL'].value_counts(dropna=False)

BEL ZONAL 1    1284
BEL ZONAL 3    1204
Name: ZONAL, dtype: int64

#### REVISION DE NO NULOS

In [19]:
data[data[col_target].notnull()].shape, data.shape

((1778, 2661), (2488, 2661))

In [20]:
nn = data[data[col_target].notnull()].isnull().sum()
no_nulos = list(nn[nn == 0].index)
len(no_nulos)

442

In [21]:
no_nulos.remove('PERIODO') 
no_nulos.remove('COD_UNICO')

## Peso de la evidencia  / informacion de su valor

In [22]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.max_columns", 1000)

In [23]:
#iv_yoy = sc.iv(
#    data[data[col_target].notnull()].drop(
#        [col for col in data.columns if 'target' in col and col != col_target] + ['COD_UNICO', 'PERIODO'], axis=1), y=col_target
#)

In [24]:
#select = iv_yoy[
#    (iv_yoy['info_value'] > 0.1) #&  (iv_yoy['info_value'] <= 1)
#]
#select.shape

In [25]:
#dict_iv = select.set_index('variable').to_dict().get('info_value', {})

In [26]:
#select[:55]

In [27]:
#select[55:110]

In [28]:
#select[110:165]

In [29]:
#select[165:220]

In [30]:
#select[220:300]

In [31]:
for cc in ['RFMClass_quartil', 'FEC_INGRESO'] + [col for col in data.columns if 'listado' in col.lower()]:
    if cc in data.columns:
        del data[cc]
    
    if cc in no_nulos:
        no_nulos.remove(cc)

## >> BINS POR WOE/ IV

In [32]:
%%time
cortes = sc.woebin(
    data[
        (data[col_target].notnull())
    ].drop(
        [col for col in data.columns if 'target' in col and col != col_target] + col_no_review, 
        axis=1
    ),
    y=col_target,
    method='tree',
    no_cores=4,
    count_distr_limit=0.05  # el 5% por bin es lo mínimo recomendable para arboles
)

[INFO] creating woe binning ...
Binning on 1778 rows and 2538 columns in 00:18:31
Wall time: 18min 31s


In [33]:
dic_maxprob_corte = {}
for k, v in cortes.items():
    #print(k, type(v))
    ##print(">"*20, ' ', k.upper())
    #display(v)
    #dic_maxprob_corte[k] = v['badprob'].max()
    max_probabilidad_malo = v[v['bin'] != 'missing']['badprob'].max()
    informacion_valor = v['total_iv'].max()
    dic_maxprob_corte[k] = [
        max_probabilidad_malo, informacion_valor, (max_probabilidad_malo + informacion_valor) / 2
    ]

In [271]:
listado_mayor_prob_corte = sorted([(k, v) for k, v in dic_maxprob_corte.items()], key=lambda _: _[1][1], reverse=True)
listado_mayor_prob_corte[:25]

[('FM_category_septil',
  [0.1141732283464567, 0.7004973117961001, 0.4073352700712784]),
 ('maximo_percent_others_u12m',
  [0.10629067245119306, 0.5345763329374138, 0.32043350269430343]),
 ('DEBT_IN_SF_ULT_RCC_y',
  [0.11475409836065574, 0.5239806068456281, 0.3193673526031419]),
 ('MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF',
  [0.11443661971830986, 0.4419280174033771, 0.27818231856084347]),
 ('DEBT_IN_SF_ULT_RCC_x',
  [0.11443661971830986, 0.4419280174033771, 0.27818231856084347]),
 ('MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF',
  [0.11443661971830986, 0.4419280174033771, 0.27818231856084347]),
 ('DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V',
  [0.11443661971830986, 0.4419280174033771, 0.27818231856084347]),
 ('FEVE', [0.11506849315068493, 0.43943309873550873, 0.2772507959430968]),
 ('MONTO_total__U2M',
  [0.11753731343283583, 0.4346315303878363, 0.27608442191033605]),
 ('RIESGO_DIRECTO_CIERRE_MES',
  [0.16981132075471697, 0.43356411316318977, 0.30168771695895336]),
 ('DEBT_PORM_u6m_anio_a

In [35]:
vars_top = [_[0] for _ in listado_mayor_prob_corte[:200]]
len(vars_top)

200

In [36]:
if 'FEC_INGRESO' in no_nulos:
    no_nulos.remove('FEC_INGRESO')

## Correlacion lineal

In [37]:
data_corr = data[data[col_target].notnull()][no_nulos].corr()

In [38]:
data_corr[col_target].sort_values(ascending=False).apply(lambda _: abs(_))[:58]

target_variante_atraso_feve_3_1Mbueno                    1.000000
target_fusion_atraso_feve_3_1Mbueno                      1.000000
target_FINAL_fusion_atraso_feve_2Mbueno                  1.000000
target_SF_8_dias_atraso_o_mas_1Mbueno                    0.859108
target_SF_8_dias_atraso_o_mas_2Mbueno                    0.859108
target_fusion_atraso_15_feve_all_2Mbueno                 0.807836
target_rcc_8_dias_atraso_2Mbueno                         0.715638
target_rcc_8_dias_atraso_1Mbueno                         0.715638
target_SF_15_dias_atraso_o_mas_2Mbueno                   0.633656
target_FINAL_fusion_atraso_SEGUIDO_15000_FEVE_2Mbueno    0.573255
target_rcc_15_dias_atraso_2Mbueno                        0.506350
target_reducir_salir_seguircovid_join_2Mbueno            0.497401
target_seguircovid_reducir_salir_1Mbueno                 0.497401
target_seguircovid_reducir_salir_2Mbueno                 0.497401
cuasi_target_preocupa                                    0.497401
target_red

In [39]:
#[col for col in data.columns if 'TOTAL' in col.upper()]

In [40]:
#select_1.to_excel("VARIABLES_CANDIDATAS_SEGUIR_COVID.xlsx", index=False)

### NUEVAS VARIABLES

Aplicamos los cortes a toda la data

In [41]:
data = all_data.copy()
del all_data

In [42]:
#from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score
#data_valid = data[data[col_target].notnull()]
#nuevas = []

In [43]:
[ col for col in data.columns if 'NO_IBK' in col.upper()]

['MAYOR_GARANTIA_AUTO_HIPO_BANCO_NO_IBK',
 'TIENE_MAS_80PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK',
 'TIENE_MAS_85PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK',
 'TIENE_MAS_95PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK',
 'TIENE_MAS_75PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK',
 'TIENE_MAS_90PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK',
 'MAYOR_GARANTIA_PREFERIDA_BANCO_NO_IBK',
 'TIENE_MAS_70PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK',
 'MAYOR_GARANTIA_Y_CAIDA_10PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF',
 'MAYOR_GARANTIA_Y_CAIDA_20PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF',
 'MAYOR_GARANTIA_Y_CAIDA_30PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF',
 'MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF',
 'NOMBRE_BANCO_MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF']

In [44]:
for column in [ col for col in data.columns if 'NO_IBK' in col.upper()]:
    display(cortes[column])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAYOR_GARANTIA_AUTO_HIPO_BANCO_NO_IBK,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.020899,missing,True
1,MAYOR_GARANTIA_AUTO_HIPO_BANCO_NO_IBK,"[-inf,1.0)",765,0.430259,719,46,0.060131,-0.010772,0.000050,0.020899,1.0,False
2,MAYOR_GARANTIA_AUTO_HIPO_BANCO_NO_IBK,"[1.0,inf)",896,0.503937,838,58,0.064732,0.067873,0.002392,0.020899,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_MAS_80PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.117185,missing,True
1,TIENE_MAS_80PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[-inf,1.0)",1241,0.697975,1151,90,0.072522,0.189871,0.027363,0.117185,1.0,False
2,TIENE_MAS_80PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[1.0,inf)",420,0.236220,406,14,0.033333,-0.628848,0.071364,0.117185,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_MAS_85PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.155403,missing,True
1,TIENE_MAS_85PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[-inf,1.0)",1311,0.737345,1216,95,0.072464,0.189003,0.028631,0.155403,1.0,False
2,TIENE_MAS_85PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[1.0,inf)",350,0.196850,341,9,0.025714,-0.896210,0.108314,0.155403,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_MAS_95PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.113413,missing,True
1,TIENE_MAS_95PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[-inf,1.0)",1420,0.798650,1322,98,0.069014,0.136514,0.015807,0.113413,1.0,False
2,TIENE_MAS_95PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[1.0,inf)",241,0.135546,235,6,0.024896,-0.929378,0.079149,0.113413,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_MAS_75PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.085894,missing,True
1,TIENE_MAS_75PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[-inf,1.0)",1183,0.665354,1098,85,0.071851,0.179853,0.023300,0.085894,1.0,False
2,TIENE_MAS_75PERCENT_GAR_TOTAL_EN_UN_SOLO_BANCO_PRIN_NO_IBK,"[1.0,inf)",478,0.268841,459,19,0.039749,-0.446164,0.044136,0.085894,inf,False


...

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAYOR_GARANTIA_Y_CAIDA_10PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF,"[-inf,1.0)",1613,0.907199,1515,98,0.060756,0.000244,5.421136e-08,5.847826e-07,1.0,False
1,MAYOR_GARANTIA_Y_CAIDA_10PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF,"[1.0,inf)",165,0.092801,155,10,0.060606,-0.002392,5.305712e-07,5.847826e-07,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAYOR_GARANTIA_Y_CAIDA_20PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF,"[-inf,1.0)",1689,0.949944,1587,102,0.060391,-0.006180,0.000036,0.000688,1.0,False
1,MAYOR_GARANTIA_Y_CAIDA_20PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF,"[1.0,inf)",89,0.050056,83,6,0.067416,0.111367,0.000652,0.000688,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAYOR_GARANTIA_Y_CAIDA_30PERCENT_DEBT_COLDIRECT_U3M_NO_IBK_SF,"[-inf,inf)",1778,1.0,1670,108,0.060742,0.0,0.0,0.0,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF,"[-inf,1.0)",1337,0.751969,1248,89,0.066567,0.097786,0.007507,0.035306,1.0,False
1,MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF,"[1.0,inf)",441,0.248031,422,19,0.043084,-0.362119,0.027799,0.035306,inf,False


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NOMBRE_BANCO_MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF,missing,1337,0.751969,1248,89,0.066567,0.097786,0.007507,0.072851,missing,True
1,NOMBRE_BANCO_MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF,BBVA,133,0.074803,130,3,0.022556,-1.030474,0.051592,0.072851,BBVA,False
2,NOMBRE_BANCO_MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF,BCP,130,0.073116,125,5,0.038462,-0.480428,0.013718,0.072851,BCP,False
3,NOMBRE_BANCO_MAYOR_GARANTIA_Y_MENOR_DEBT_COLDIRECT_U3M_BANK_NO_IBK_SF,"SCOTIA%,%BIF",178,0.100112,167,11,0.061798,0.018349,0.000034,0.072851,"SCOTIA%,%BIF",False


In [45]:
var = vars_top[0]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FM_category_septil


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FM_category_septil,"_62%,%_41%,%_36%,%_34%,%_57%,%_61%,%_51%,%_26%,%_25%,%_47%,%_63%,%_71%,%_73%,%_75%,%_76%,%_53",278,0.156355,277,1,0.003597,-2.885570,0.451906,0.700497,"_62%,%_41%,%_36%,%_34%,%_57%,%_61%,%_51%,%_26%,%_25%,%_47%,%_63%,%_71%,%_73%,%_75%,%_76%,%_53",False
1,FM_category_septil,"_65%,%_24%,%_54%,%nulo%,%_56%,%_14%,%_35",294,0.165354,285,9,0.030612,-0.716817,0.062596,0.700497,"_65%,%_24%,%_54%,%nulo%,%_56%,%_14%,%_35",False
2,FM_category_septil,"_11%,%_13%,%_45%,%_66%,%_77%,%_42%,%_31%,%_21%,%_32%,%_55%,%_46",698,0.392576,658,40,0.057307,-0.061878,0.001463,0.700497,"_11%,%_13%,%_45%,%_66%,%_77%,%_42%,%_31%,%_21%,%_32%,%_55%,%_46",False
3,FM_category_septil,"_43%,%_12%,%_23%,%_33%,%_74%,%_52%,%_44%,%_22%,%_64%,%_67%,%_15",508,0.285714,450,58,0.114173,0.689643,0.184532,0.700497,"_43%,%_12%,%_23%,%_33%,%_74%,%_52%,%_44%,%_22%,%_64%,%_67%,%_15",False


In [46]:
data = add_new_var_x_cut_up(data, colum='DEBT_PORM_u6m_anio_act', cortes=[14000000.0, 22000000.0, 42000000.0], impute=0, up=True, soft=False)

>>  _14000000.0_cut_up_DEBT_PORM_u6m_anio_act
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.35233644859813085  PRECISION:  0.06696269982238011  F1:  0.11253731343283582  GINI:  0.0532928416301246
//////////////////////////////////////////////////
>>  _22000000.0_cut_up_DEBT_PORM_u6m_anio_act
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2691588785046729  PRECISION:  0.06666666666666667  F1:  0.10686456400742116  GINI:  0.0396245508262032
//////////////////////////////////////////////////
>>  _42000000.0_cut_up_DEBT_PORM_u6m_anio_act
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.14579439252336449  PRECISION:  0.0611764705882353  F1:  0.0861878453038674  GINI:  0.009508385464273017
//////////////////////////////////////////////////


In [47]:
var = vars_top[1]
print(">>>>> ", var)
display(cortes[var])

>>>>>  maximo_percent_others_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,maximo_percent_others_u12m,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.534576,missing,True
1,maximo_percent_others_u12m,"[-inf,0.8)",461,0.259280,412,49,0.106291,0.609245,0.126112,0.534576,0.8,False
2,maximo_percent_others_u12m,"[0.8,0.9)",187,0.105174,173,14,0.074866,0.224213,0.005838,0.534576,0.9,False
3,maximo_percent_others_u12m,"[0.9,1.0)",770,0.433071,730,40,0.051948,-0.165717,0.011063,0.534576,1.0,False
4,maximo_percent_others_u12m,"[1.0,inf)",243,0.136670,242,1,0.004115,-2.750490,0.373107,0.534576,inf,False


In [48]:
var = vars_top[2]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN_SF_ULT_RCC_y


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN_SF_ULT_RCC_y,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.523981,missing,True
1,DEBT_IN_SF_ULT_RCC_y,"[-inf,6000000.0)",331,0.186164,314,17,0.051360,-0.186031,0.005938,0.523981,6000000.0,False
2,DEBT_IN_SF_ULT_RCC_y,"[6000000.0,16000000.0)",509,0.286277,493,16,0.031434,-0.697772,0.103470,0.523981,16000000.0,False
3,DEBT_IN_SF_ULT_RCC_y,"[16000000.0,22000000.0)",229,0.128796,224,5,0.021834,-1.072059,0.094575,0.523981,22000000.0,False
4,DEBT_IN_SF_ULT_RCC_y,"[22000000.0,inf)",610,0.343082,540,70,0.114754,0.687075,0.219478,0.523981,inf,False


In [49]:
_text = 'TURISMO Y HOSPEDAJE%,%RESTAURANTES%,%AUTOMOTRIZ VEHÍCULOS%,%TEXTIL'

data['sei_letal_gran_empresa'] = data['SEI_MCDO'].apply(lambda _: _.upper() in _text.upper())

In [50]:
var = vars_top[3]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.441928,missing,True
1,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[-inf,4000000.0)",231,0.129921,218,13,0.056277,-0.089397,0.000998,0.441928,4000000.0,False
2,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[4000000.0,24000000.0)",885,0.497750,855,30,0.033898,-0.619755,0.146568,0.441928,24000000.0,False
3,MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF,"[24000000.0,inf)",568,0.319460,503,65,0.114437,0.683946,0.202229,0.441928,inf,False


In [273]:
data = add_new_var_x_cut_up(data, colum='MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF', cortes=[20000000 , 24000000, 30000000], impute=0, up=True, soft=False)

>>  _20000000_cut_up_MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.18317757009345795  PRECISION:  0.07091172214182344  F1:  0.10224308815858112  GINI:  0.03698606377443259
//////////////////////////////////////////////////
>>  _24000000_cut_up_MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.15700934579439252  PRECISION:  0.07288503253796096  F1:  0.09955555555555555  GINI:  0.03535387499853693
//////////////////////////////////////////////////
>>  _30000000_cut_up_MTO_DEUDA_VIGENTE_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11775700934579439  PRECISION:  0.06766917293233082  F1:  0.0859481582537517  GINI:  0.018929729373119875
//////////////////////////////////////////////////


In [51]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_SF', cortes=[2,3,4,5], impute=0, up=True, soft=False)

>>  _2_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.20101351351351351  F1:  0.14320096269554755  GINI:  0.08428793516787425
//////////////////////////////////////////////////
>>  _3_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08598130841121496  PRECISION:  0.22493887530562348  F1:  0.1244083840432725  GINI:  0.06793508274800186
//////////////////////////////////////////////////
>>  _4_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06542056074766354  PRECISION:  0.24305555555555555  F1:  0.10309278350515463  GINI:  0.053010222594412904
//////////////////////////////////////////////////
>>  _5_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04579439252336449  PRECISION:  0.2474747474747475  F1:  0.07728706624605677  GINI:  0.03731209718008777
////////////////////////////////////////

In [52]:
var = vars_top[4]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN_SF_ULT_RCC_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN_SF_ULT_RCC_x,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.441928,missing,True
1,DEBT_IN_SF_ULT_RCC_x,"[-inf,4000000.0)",231,0.129921,218,13,0.056277,-0.089397,0.000998,0.441928,4000000.0,False
2,DEBT_IN_SF_ULT_RCC_x,"[4000000.0,24000000.0)",885,0.497750,855,30,0.033898,-0.619755,0.146568,0.441928,24000000.0,False
3,DEBT_IN_SF_ULT_RCC_x,"[24000000.0,inf)",568,0.319460,503,65,0.114437,0.683946,0.202229,0.441928,inf,False


In [53]:
data = add_new_var_x_cut_up(data, colum='PERCENT_SEI_VPC_NO_CLASIF_NORMAL', cortes=[0.39], impute=0, up=True, soft=False)

>>  _0.39_cut_up_PERCENT_SEI_VPC_NO_CLASIF_NORMAL
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0  PRECISION:  0.0  F1:  0.0  GINI:  0.0
//////////////////////////////////////////////////


In [54]:
var = vars_top[5]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.441928,missing,True
1,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[-inf,4000000.0)",231,0.129921,218,13,0.056277,-0.089397,0.000998,0.441928,4000000.0,False
2,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[4000000.0,24000000.0)",885,0.497750,855,30,0.033898,-0.619755,0.146568,0.441928,24000000.0,False
3,MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF,"[24000000.0,inf)",568,0.319460,503,65,0.114437,0.683946,0.202229,0.441928,inf,False


In [55]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1560747663551402  PRECISION:  0.2014475271411339  F1:  0.17588204318062142  GINI:  0.11838832664205823
//////////////////////////////////////////////////


In [56]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF', cortes=[2, 3], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09252336448598131  PRECISION:  0.24688279301745636  F1:  0.13460231135282122  GINI:  0.07533106117276245
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.05794392523364486  PRECISION:  0.29523809523809524  F1:  0.09687500000000002  GINI:  0.04951855804703453
//////////////////////////////////////////////////


In [57]:
var = vars_top[6]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.441928,missing,True
1,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[-inf,4000000.0)",231,0.129921,218,13,0.056277,-0.089397,0.000998,0.441928,4000000.0,False
2,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[4000000.0,24000000.0)",885,0.497750,855,30,0.033898,-0.619755,0.146568,0.441928,24000000.0,False
3,DEUDA_DIRECTA_TOTAL_VIG_VENC_LIN_MER_V,"[24000000.0,inf)",568,0.319460,503,65,0.114437,0.683946,0.202229,0.441928,inf,False


In [58]:
from datetime import datetime

In [59]:
data['cambio_accionistas_o_cambio_gerencia'] = data[['CAMBIO_ACCIONISTAS_ANNIO', 'CAMBIO_GERENCIA_GENERAL_ANNIO']].apply(
    lambda _: _[0] == datetime.now().year or _[1] == datetime.now().year, axis=1
).astype(int)

pd.crosstab(
    data['cambio_accionistas_o_cambio_gerencia'], data[col_target].fillna(0)
)

target_FINAL_fusion_atraso_feve_2Mbueno,0.0,1.0
cambio_accionistas_o_cambio_gerencia,,
0,22944,1070


In [60]:
var = vars_top[7]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FEVE


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FEVE,missing,644,0.362205,631,13,0.020186,-1.143909,0.294527,0.439433,missing,True
1,FEVE,EX-FEVE,769,0.432508,716,53,0.068921,0.135059,0.008373,0.439433,EX-FEVE,False
2,FEVE,"GARANTIZAR%,%SEGUIR",365,0.205287,323,42,0.115068,0.698465,0.136533,0.439433,"GARANTIZAR%,%SEGUIR",False


In [274]:
data = pd.get_dummies(data, dummy_na=True, columns=['FEVE'])

In [61]:
data = add_new_var_x_cut_up(data, colum='ROTACION_CTASXPAGAR_CEF_SITUACION', cortes=[85,95,110], impute=0, up=True, soft=False)

>>  _85_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.27476635514018694  PRECISION:  0.06970128022759602  F1:  0.11119515885022693  GINI:  0.051380268381675975
//////////////////////////////////////////////////
>>  _95_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.23551401869158878  PRECISION:  0.06902218570254724  F1:  0.10675704299936453  GINI:  0.04201521418287868
//////////////////////////////////////////////////
>>  _110_cut_up_ROTACION_CTASXPAGAR_CEF_SITUACION
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.17943925233644858  PRECISION:  0.06372386325921009  F1:  0.0940484937545922  GINI:  0.018844922380852447
//////////////////////////////////////////////////


In [62]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO', cortes=[1, 2, 3, 4, 5], impute=0, up=True, soft=False)

>>  _1_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1542056074766355  PRECISION:  0.1966626936829559  F1:  0.17286537454164483  GINI:  0.1158360298835579
//////////////////////////////////////////////////
>>  _2_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11401869158878504  PRECISION:  0.21631205673758866  F1:  0.1493268053855569  GINI:  0.08885644634228629
//////////////////////////////////////////////////
>>  _3_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09439252336448598  PRECISION:  0.2433734939759036  F1:  0.13602693602693602  GINI:  0.07651708217127173
//////////////////////////////////////////////////
>>  _4_cut_up_MAX_DIAS_ATRASO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06822429906542056  PRECISION:  0.2616487455197133  F1:  0.10822831727205337  GINI:  0.05649709879216536
//////////////////////////////////////////////////
>>  _5

In [63]:
var = vars_top[8]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_total__U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_total__U2M,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.434632,missing,True
1,MONTO_total__U2M,"[-inf,4000000.0)",234,0.131609,221,13,0.055556,-0.103064,0.001336,0.434632,4000000.0,False
2,MONTO_total__U2M,"[4000000.0,26000000.0)",921,0.517998,889,32,0.034745,-0.594212,0.141712,0.434632,26000000.0,False
3,MONTO_total__U2M,"[26000000.0,inf)",536,0.301462,473,63,0.117537,0.714188,0.210885,0.434632,inf,False


In [64]:
var = vars_top[9]
print(">>>>> ", var)
display(cortes[var])

>>>>>  RIESGO_DIRECTO_CIERRE_MES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,RIESGO_DIRECTO_CIERRE_MES,missing,131,0.073678,130,1,0.007634,-2.129087,0.146024,0.433564,missing,True
1,RIESGO_DIRECTO_CIERRE_MES,"[-inf,450000.0)",1052,0.591676,1006,46,0.043726,-0.346648,0.061173,0.433564,450000.0,False
2,RIESGO_DIRECTO_CIERRE_MES,"[450000.0,1500000.0)",343,0.192913,314,29,0.084548,0.356351,0.028684,0.433564,1500000.0,False
3,RIESGO_DIRECTO_CIERRE_MES,"[1500000.0,2700000.0)",93,0.052306,88,5,0.053763,-0.129451,0.000828,0.433564,2700000.0,False
4,RIESGO_DIRECTO_CIERRE_MES,"[2700000.0,inf)",159,0.089426,132,27,0.169811,1.151483,0.196855,0.433564,inf,False


In [276]:
data = add_new_var_x_cut_up(data, colum='RIESGO_DIRECTO_CIERRE_MES', cortes=[1000000, 1500000, 2000000, 2500000, 2700000], impute=0, up=True, soft=False)

>>  _1000000_cut_up_RIESGO_DIRECTO_CIERRE_MES
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11682242990654206  PRECISION:  0.07516536380036079  F1:  0.09147457006952069  GINI:  0.02926692495379246
//////////////////////////////////////////////////
>>  _1500000_cut_up_RIESGO_DIRECTO_CIERRE_MES
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08598130841121496  PRECISION:  0.07082371054657428  F1:  0.07766990291262135  GINI:  0.01726902331500635
//////////////////////////////////////////////////
>>  _2000000_cut_up_RIESGO_DIRECTO_CIERRE_MES
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.07850467289719626  PRECISION:  0.07407407407407407  F1:  0.07622504537205081  GINI:  0.018730108397594813
//////////////////////////////////////////////////
>>  _2500000_cut_up_RIESGO_DIRECTO_CIERRE_MES
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06728971962616823  PRECISION:  0.07178464606181456  F1:  0.0694645441389291  GINI:  0.01428

In [65]:
var = vars_top[10]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_PORM_u6m_anio_act


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_PORM_u6m_anio_act,missing,49,0.027559,49,0,0.000000,-1.267032,0.026705,0.431193,missing,True
1,DEBT_PORM_u6m_anio_act,"[-inf,5000000.0)",197,0.110799,186,11,0.055838,-0.097703,0.001013,0.431193,5000000.0,False
2,DEBT_PORM_u6m_anio_act,"[5000000.0,35000000.0)",747,0.420135,725,22,0.029451,-0.764980,0.177561,0.431193,35000000.0,False
3,DEBT_PORM_u6m_anio_act,"[35000000.0,40000000.0)",106,0.059618,96,10,0.094340,0.468386,0.016085,0.431193,40000000.0,False
4,DEBT_PORM_u6m_anio_act,"[40000000.0,55000000.0)",237,0.133296,226,11,0.046414,-0.292491,0.010038,0.431193,55000000.0,False
5,DEBT_PORM_u6m_anio_act,"[55000000.0,inf)",442,0.248594,388,54,0.122172,0.758128,0.199791,0.431193,inf,False


In [66]:
var = vars_top[11]
print(">>>>> ", var)
display(cortes[var])

>>>>>  maximo_percent_sow_others_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,maximo_percent_sow_others_u12m,missing,18,0.010124,18,0,0.000000,-0.265583,0.000668,0.426332,missing,True
1,maximo_percent_sow_others_u12m,"[-inf,0.5)",284,0.159730,258,26,0.091549,0.435286,0.036677,0.426332,0.5,False
2,maximo_percent_sow_others_u12m,"[0.5,0.7000000000000001)",163,0.091676,143,20,0.122699,0.763037,0.074797,0.426332,0.7000000000000001,False
3,maximo_percent_sow_others_u12m,"[0.7000000000000001,1.0)",912,0.512936,855,57,0.062500,0.022099,0.000253,0.426332,1.0,False
4,maximo_percent_sow_others_u12m,"[1.0,inf)",401,0.225534,396,5,0.012469,-1.641827,0.313937,0.426332,inf,False


In [67]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_SIN_LINEA_VIGENTE', cortes=[1,2,3,4], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3037383177570093  PRECISION:  0.08807588075880758  F1:  0.13655462184873948  GINI:  0.1121750705749529
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2644859813084112  PRECISION:  0.0874806800618238  F1:  0.13147502903600464  GINI:  0.09643406282953149
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22429906542056074  PRECISION:  0.08456659619450317  F1:  0.12282497441146367  GINI:  0.07639971440154691
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.17757009345794392  PRECISION:  0.07841518778373917  F1:  0.10878900658459778  GINI:  0.05044951962212463
///////

In [68]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GRUPO_ECON_VPC_SEGUIR', cortes=[0.1, 0.2, 0.3], impute=0, up=True, soft=False)

>>  _0.1_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06355140186915888  PRECISION:  0.0787037037037037  F1:  0.0703205791106515  GINI:  0.018236589162794248
//////////////////////////////////////////////////
>>  _0.2_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.024299065420560748  PRECISION:  0.07027027027027027  F1:  0.036111111111111115  GINI:  0.004715779527358022
//////////////////////////////////////////////////
>>  _0.3_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.010280373831775701  PRECISION:  0.0763888888888889  F1:  0.01812191103789127  GINI:  0.0027089289951593987
//////////////////////////////////////////////////


In [69]:
var = vars_top[12]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_OTROS_BANCOS_PRIN_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_OTROS_BANCOS_PRIN_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.4146,missing,True
1,MTOSALDO_OTROS_BANCOS_PRIN_V,"[-inf,2000000.0)",402,0.226097,373,29,0.072139,0.175866,0.007553,0.4146,2000000.0,False
2,MTOSALDO_OTROS_BANCOS_PRIN_V,"[2000000.0,8000000.0)",282,0.158605,277,5,0.017730,-1.284431,0.154073,0.4146,8000000.0,False
3,MTOSALDO_OTROS_BANCOS_PRIN_V,"[8000000.0,22000000.0)",576,0.323960,549,27,0.046875,-0.282113,0.022797,0.4146,22000000.0,False
4,MTOSALDO_OTROS_BANCOS_PRIN_V,"[22000000.0,52000000.0)",258,0.145107,227,31,0.120155,0.739186,0.109944,0.4146,52000000.0,False
5,MTOSALDO_OTROS_BANCOS_PRIN_V,"[52000000.0,inf)",166,0.093363,150,16,0.096386,0.492102,0.028101,0.4146,inf,False


In [277]:
data = add_new_var_x_cut_up(data, colum='MTOSALDO_OTROS_BANCOS_PRIN_V', cortes=[22000000, 52000000], impute=0, up=True, soft=False)

>>  _22000000_cut_up_MTOSALDO_OTROS_BANCOS_PRIN_V
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11869158878504672  PRECISION:  0.07211811470755253  F1:  0.08972094666195692  GINI:  0.025670980792333653
//////////////////////////////////////////////////
>>  _52000000_cut_up_MTOSALDO_OTROS_BANCOS_PRIN_V
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04579439252336449  PRECISION:  0.0546875  F1:  0.049847405900305196  GINI:  -0.002423756172980629
//////////////////////////////////////////////////


In [70]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF', cortes=[2,3,4,5], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12990654205607477  PRECISION:  0.17866323907455012  F1:  0.15043290043290042  GINI:  0.09352944994631729
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09906542056074766  PRECISION:  0.21721311475409835  F1:  0.1360718870346598  GINI:  0.077318864714226
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0691588785046729  PRECISION:  0.2379421221864952  F1:  0.10716871832005793  GINI:  0.05566690537476293
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.048598130841121495  PRECISION:  0.29714285714285715  F1:  0.0835341365461

In [71]:
data = add_new_var_x_cut_up(data, colum='maximo_percent_others_u12m', cortes=[0.6, 0.7, 0.8, 0.9], impute=0, up=True, soft=False)

>>  _0.6_cut_up_maximo_percent_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6560747663551402  PRECISION:  0.06422102277925167  F1:  0.11699025081243228  GINI:  0.07375665181568891
//////////////////////////////////////////////////
>>  _0.7_cut_up_maximo_percent_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.602803738317757  PRECISION:  0.0629513956666016  F1:  0.11399787910922589  GINI:  0.0562365061647343
//////////////////////////////////////////////////
>>  _0.8_cut_up_maximo_percent_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5242990654205607  PRECISION:  0.059833617747440275  F1:  0.1074095347501436  GINI:  0.022477364407239353
//////////////////////////////////////////////////
>>  _0.9_cut_up_maximo_percent_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4130841121495327  PRECISION:  0.05452078450721599  F1:  0.09632777596164324  GINI:  -0.02327020869755836
////

In [72]:
var = vars_top[13]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,missing,1166,0.655793,1126,40,0.034305,-0.599100,0.182055,0.412822,missing,True
1,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,"[-inf,0.1)",296,0.166479,269,27,0.091216,0.439573,0.039088,0.412822,0.1,False
2,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,"[0.1,0.9)",96,0.053993,80,16,0.166667,1.129010,0.113176,0.412822,0.9,False
3,ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO,"[0.9,inf)",220,0.123735,195,25,0.113636,0.684324,0.078502,0.412822,inf,False


In [278]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO', cortes=[0.9, 1, 1.1, 1.2, 1.3], impute=0, up=True, soft=False)

>>  _0.9_cut_up_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2205607476635514  PRECISION:  0.11184834123222749  F1:  0.14842767295597484  GINI:  0.11387738207092912
//////////////////////////////////////////////////
>>  _1_cut_up_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2177570093457944  PRECISION:  0.11148325358851674  F1:  0.14746835443037973  GINI:  0.11204142241649917
//////////////////////////////////////////////////
>>  _1.1_cut_up_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.07102803738317758  PRECISION:  0.1554192229038855  F1:  0.09749839640795381  GINI:  0.047516708680000797
//////////////////////////////////////////////////
>>  _1.2_cut_up_ATRASO_PROM_ULT_TRIM_OVER_TRIM_PASADO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06728971962616823  PRECISION:  0.15824175824175823  F1:  0.09442

In [73]:
data = add_new_var_x_cut_up(data, colum='IMPORTE_TRIB_PROMEDIO_U9M', cortes=[5000], impute=0, up=True, soft=False)

>>  _5000_cut_up_IMPORTE_TRIB_PROMEDIO_U9M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11775700934579439  PRECISION:  0.10387469084913438  F1:  0.11038107752956636  GINI:  0.05587610304954049
//////////////////////////////////////////////////


In [74]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_SF', cortes=[1,2,3,4,5], impute=0, up=True, soft=False)

>>  _1_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22242990654205608  PRECISION:  0.17921686746987953  F1:  0.19849874895746455  GINI:  0.16037821577580313
//////////////////////////////////////////////////
>>  _2_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.20101351351351351  F1:  0.14320096269554755  GINI:  0.08428793516787425
//////////////////////////////////////////////////
>>  _3_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08598130841121496  PRECISION:  0.22493887530562348  F1:  0.1244083840432725  GINI:  0.06793508274800186
//////////////////////////////////////////////////
>>  _4_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06542056074766354  PRECISION:  0.24305555555555555  F1:  0.10309278350515463  GINI:  0.053010222594412904
///////////////////////////////////////

In [75]:

texto_a_comparar = """
Servicios diversos%,%Saneamiento%,%Artículos diversos%,%Real Estate%,%Vehículos%,%Alquiler de equipos%,%Plástico y derivados%,%Textil y calzado%,%Químicos%,%Ganadero%,%Artículos de oficina%,%Subcontratistas mineros%,%Vehículos, repuestos y accesorios%,%Restaurante y eventos sociales	
""".upper()

data['cut_letal_SUBSECTOR_gran_empresa'] = data['SUBSECTOR'].apply(lambda _: None if pd.isnull(_) else (1 if _.upper() in texto_a_comparar.upper() else 0))



In [76]:
var = vars_top[14]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COL_DIRECTA_PROMEDIO_MENSUAL_U3M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COL_DIRECTA_PROMEDIO_MENSUAL_U3M,missing,242,0.136108,235,7,0.028926,-0.775228,0.058843,0.404683,missing,True
1,COL_DIRECTA_PROMEDIO_MENSUAL_U3M,"[-inf,1000000.0)",855,0.480877,823,32,0.037427,-0.508773,0.099983,0.404683,1000000.0,False
2,COL_DIRECTA_PROMEDIO_MENSUAL_U3M,"[1000000.0,3800000.0)",396,0.222722,361,35,0.088384,0.404918,0.043693,0.404683,3800000.0,False
3,COL_DIRECTA_PROMEDIO_MENSUAL_U3M,"[3800000.0,6600000.0)",118,0.066367,112,6,0.050847,-0.188292,0.002167,0.404683,6600000.0,False
4,COL_DIRECTA_PROMEDIO_MENSUAL_U3M,"[6600000.0,inf)",167,0.093926,139,28,0.167665,1.136178,0.199997,0.404683,inf,False


In [77]:
var = vars_top[15]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEUDA_TOTAL_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEUDA_TOTAL_RCC,"[-inf,22000000.0)",1117,0.628234,1079,38,0.034020,-0.607756,0.178836,0.38779,22000000.0,False
1,DEUDA_TOTAL_RCC,"[22000000.0,24000000.0)",93,0.052306,88,5,0.053763,-0.129451,0.000828,0.38779,24000000.0,False
2,DEUDA_TOTAL_RCC,"[24000000.0,inf)",568,0.319460,503,65,0.114437,0.692245,0.208126,0.38779,inf,False


In [279]:
data = add_new_var_x_cut_up(data, colum='DEUDA_TOTAL_RCC', cortes=[14000000, 20000000, 24000000], impute=0, up=True, soft=False)

>>  _14000000_cut_up_DEUDA_TOTAL_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2514018691588785  PRECISION:  0.0697433238268084  F1:  0.10919423584331236  GINI:  0.047143643040240324
//////////////////////////////////////////////////
>>  _20000000_cut_up_DEUDA_TOTAL_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.18317757009345795  PRECISION:  0.07091172214182344  F1:  0.10224308815858112  GINI:  0.03698606377443259
//////////////////////////////////////////////////
>>  _24000000_cut_up_DEUDA_TOTAL_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.15700934579439252  PRECISION:  0.07288503253796096  F1:  0.09955555555555555  GINI:  0.03535387499853693
//////////////////////////////////////////////////


In [78]:
[col for col in data.columns if 'inventarios' in col.lower()]

['ROTACION_INVENTARIOS_CEF_SITUACION',
 'ROTACION_INVENTARIOS_VAR_DIAS_CEF_SITUACION',
 'ROTACION_INVENTARIOS_VAR_CEF_SITUACION']

In [79]:
var = vars_top[16]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTDO_COL_DIRECTAS_SF_1UM


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTDO_COL_DIRECTAS_SF_1UM,missing,49,0.027559,49,0,0.000000,-1.267032,0.026705,0.387613,missing,True
1,MTDO_COL_DIRECTAS_SF_1UM,"[-inf,6000000.0)",373,0.209786,356,17,0.045576,-0.311569,0.017780,0.387613,6000000.0,False
2,MTDO_COL_DIRECTAS_SF_1UM,"[6000000.0,24000000.0)",788,0.443195,762,26,0.032995,-0.647701,0.140898,0.387613,24000000.0,False
3,MTDO_COL_DIRECTAS_SF_1UM,"[24000000.0,inf)",568,0.319460,503,65,0.114437,0.683946,0.202229,0.387613,inf,False


In [80]:
var = vars_top[17]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COL_DIRECTA_MES_ACTUAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COL_DIRECTA_MES_ACTUAL,missing,242,0.136108,235,7,0.028926,-0.775228,0.058843,0.379418,missing,True
1,COL_DIRECTA_MES_ACTUAL,"[-inf,1000000.0)",884,0.497188,848,36,0.040724,-0.420914,0.073429,0.379418,1000000.0,False
2,COL_DIRECTA_MES_ACTUAL,"[1000000.0,3600000.0)",370,0.208099,340,30,0.081081,0.310699,0.023049,0.379418,3600000.0,False
3,COL_DIRECTA_MES_ACTUAL,"[3600000.0,6200000.0)",95,0.053431,91,4,0.042105,-0.386117,0.006739,0.379418,6200000.0,False
4,COL_DIRECTA_MES_ACTUAL,"[6200000.0,inf)",187,0.105174,156,31,0.165775,1.122579,0.217358,0.379418,inf,False


In [81]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN', cortes=[0, -19], impute=0, up=False, soft=True)

>>  _0_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4261682242990654  PRECISION:  0.06919575113808801  F1:  0.11906005221932114  GINI:  0.07697091130806011
//////////////////////////////////////////////////
>>  _-19_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04953271028037383  PRECISION:  0.11300639658848614  F1:  0.06887589343729694  GINI:  0.02585059710719828
//////////////////////////////////////////////////


In [82]:
var = vars_top[18]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COL_DIRECTA_PROMEDIO_MENSUAL_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COL_DIRECTA_PROMEDIO_MENSUAL_U1M,missing,242,0.136108,235,7,0.028926,-0.775228,0.058843,0.379418,missing,True
1,COL_DIRECTA_PROMEDIO_MENSUAL_U1M,"[-inf,1000000.0)",884,0.497188,848,36,0.040724,-0.420914,0.073429,0.379418,1000000.0,False
2,COL_DIRECTA_PROMEDIO_MENSUAL_U1M,"[1000000.0,3600000.0)",370,0.208099,340,30,0.081081,0.310699,0.023049,0.379418,3600000.0,False
3,COL_DIRECTA_PROMEDIO_MENSUAL_U1M,"[3600000.0,6200000.0)",95,0.053431,91,4,0.042105,-0.386117,0.006739,0.379418,6200000.0,False
4,COL_DIRECTA_PROMEDIO_MENSUAL_U1M,"[6200000.0,inf)",187,0.105174,156,31,0.165775,1.122579,0.217358,0.379418,inf,False


In [83]:
var = vars_top[19]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_FULL_RCC_IBK


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_FULL_RCC_IBK,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.36275,missing,True
1,MTO_DEUDA_FULL_RCC_IBK,"[-inf,2000000.0)",598,0.336333,581,17,0.028428,-0.801389,0.153704,0.36275,2000000.0,False
2,MTO_DEUDA_FULL_RCC_IBK,"[2000000.0,4000000.0)",104,0.058493,92,12,0.115385,0.693267,0.038201,0.36275,4000000.0,False
3,MTO_DEUDA_FULL_RCC_IBK,"[4000000.0,20000000.0)",469,0.263780,444,25,0.053305,-0.146800,0.005329,0.36275,20000000.0,False
4,MTO_DEUDA_FULL_RCC_IBK,"[20000000.0,52000000.0)",468,0.263217,436,32,0.068376,0.118243,0.003875,0.36275,52000000.0,False
5,MTO_DEUDA_FULL_RCC_IBK,"[52000000.0,inf)",138,0.077615,116,22,0.159420,1.067601,0.141520,0.36275,inf,False


In [84]:
var = vars_top[20]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_IBK_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_IBK_V,"[-inf,500000.0)",645,0.362767,625,20,0.031008,-0.703572,0.133022,0.3613,500000.0,False
1,MTOSALDO_IBK_V,"[500000.0,6000000.0)",574,0.322835,535,39,0.067944,0.119743,0.004880,0.3613,6000000.0,False
2,MTOSALDO_IBK_V,"[6000000.0,8500000.0)",120,0.067492,118,2,0.016667,-1.339090,0.069820,0.3613,8500000.0,False
3,MTOSALDO_IBK_V,"[8500000.0,18000000.0)",341,0.191789,310,31,0.090909,0.435863,0.044200,0.3613,18000000.0,False
4,MTOSALDO_IBK_V,"[18000000.0,inf)",98,0.055118,82,16,0.163265,1.104317,0.109379,0.3613,inf,False


In [280]:
data = add_new_var_x_cut_up(data, colum='MTOSALDO_IBK_V', cortes=[18000000, 8500000], impute=0, up=True, soft=False)

>>  _18000000_cut_up_MTOSALDO_IBK_V
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04766355140186916  PRECISION:  0.07015130674002751  F1:  0.056761268781302165  GINI:  0.009180117495459106
//////////////////////////////////////////////////
>>  _8500000_cut_up_MTOSALDO_IBK_V
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.13738317757009347  PRECISION:  0.07146329606222654  F1:  0.09401982731052126  GINI:  0.02865039833748506
//////////////////////////////////////////////////


In [85]:
var = vars_top[21]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_SFSF_ULT_RCC_y


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_SFSF_ULT_RCC_y,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.359771,missing,True
1,MAX_DIAS_ATRASO_SFSF_ULT_RCC_y,"[-inf,2.0)",1585,0.891451,1499,86,0.054259,-0.128057,0.013816,0.359771,2.0,False
2,MAX_DIAS_ATRASO_SFSF_ULT_RCC_y,"[2.0,inf)",94,0.052868,72,22,0.234043,1.544525,0.245435,0.359771,inf,False


In [86]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GRUPO_ECON_VPC_SEGUIR', cortes=[0.2, 0.3], impute=0, up=True, soft=False)

>>  _0.2_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.024299065420560748  PRECISION:  0.07027027027027027  F1:  0.036111111111111115  GINI:  0.004715779527358022
//////////////////////////////////////////////////
>>  _0.3_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.010280373831775701  PRECISION:  0.0763888888888889  F1:  0.01812191103789127  GINI:  0.0027089289951593987
//////////////////////////////////////////////////


In [87]:
var = vars_top[22]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_SF,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.359771,missing,True
1,MAX_DIAS_ATRASO_SF,"[-inf,2.0)",1585,0.891451,1499,86,0.054259,-0.128057,0.013816,0.359771,2.0,False
2,MAX_DIAS_ATRASO_SF,"[2.0,inf)",94,0.052868,72,22,0.234043,1.544525,0.245435,0.359771,inf,False


In [281]:
data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_SF', cortes=[2, 3, 4, 5, 6, 7], impute=0, up=True, soft=False)

>>  _2_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.20101351351351351  F1:  0.14320096269554755  GINI:  0.08428793516787425
//////////////////////////////////////////////////
>>  _3_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08598130841121496  PRECISION:  0.22493887530562348  F1:  0.1244083840432725  GINI:  0.06793508274800186
//////////////////////////////////////////////////
>>  _4_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06542056074766354  PRECISION:  0.24305555555555555  F1:  0.10309278350515463  GINI:  0.053010222594412904
//////////////////////////////////////////////////
>>  _5_cut_up_MAX_DIAS_ATRASO_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04579439252336449  PRECISION:  0.2474747474747475  F1:  0.07728706624605677  GINI:  0.03731209718008777
////////////////////////////////////////

In [88]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GRUPO_ECON_VPC_SEGUIR', cortes=[0.00699, 0.009], impute=0, up=True, soft=False)

>>  _0.00699_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08691588785046729  PRECISION:  0.06330837304288632  F1:  0.07325718786923986  GINI:  0.00858274427765604
//////////////////////////////////////////////////
>>  _0.009_cut_up_PERCENT_GRUPO_ECON_VPC_SEGUIR
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08691588785046729  PRECISION:  0.06330837304288632  F1:  0.07325718786923986  GINI:  0.00858274427765604
//////////////////////////////////////////////////


In [89]:
var = vars_top[23]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PROM_CANT_TRX_U12M_CASHOUT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PROM_CANT_TRX_U12M_CASHOUT,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.359071,missing,True
1,PROM_CANT_TRX_U12M_CASHOUT,"[-inf,10.0)",297,0.167042,283,14,0.047138,-0.267942,0.010673,0.359071,10.0,False
2,PROM_CANT_TRX_U12M_CASHOUT,"[10.0,30.0)",396,0.222722,384,12,0.030303,-0.727288,0.086423,0.359071,30.0,False
3,PROM_CANT_TRX_U12M_CASHOUT,"[30.0,70.0)",448,0.251969,410,38,0.084821,0.359877,0.038270,0.359071,70.0,False
4,PROM_CANT_TRX_U12M_CASHOUT,"[70.0,110.0)",237,0.133296,228,9,0.037975,-0.493673,0.026260,0.359071,110.0,False
5,PROM_CANT_TRX_U12M_CASHOUT,"[110.0,200.0)",189,0.106299,161,28,0.148148,0.989248,0.161101,0.359071,200.0,False
6,PROM_CANT_TRX_U12M_CASHOUT,"[200.0,inf)",120,0.067492,116,4,0.033333,-0.628848,0.020390,0.359071,inf,False


In [90]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT', cortes=[0, -48], impute=0, up=False, soft=True)

>>  _0_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.44672897196261685  PRECISION:  0.06992393212404915  F1:  0.12092081963066027  GINI:  0.08477975187836306
//////////////////////////////////////////////////
>>  _-48_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.044859813084112146  PRECISION:  0.14201183431952663  F1:  0.06818181818181819  GINI:  0.028350647650889016
//////////////////////////////////////////////////


In [91]:
var = vars_top[24]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_OTROSBCOS_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_OTROSBCOS_ULT_RCC,missing,18,0.010124,18,0,0.000000,-0.265583,0.000668,0.356086,missing,True
1,SOW_OTROSBCOS_ULT_RCC,"[-inf,0.1)",236,0.132733,212,24,0.101695,0.551616,0.051543,0.356086,0.1,False
2,SOW_OTROSBCOS_ULT_RCC,"[0.1,0.4)",205,0.115298,200,5,0.024390,-0.958731,0.070799,0.356086,0.4,False
3,SOW_OTROSBCOS_ULT_RCC,"[0.4,0.6000000000000001)",215,0.120922,193,22,0.102326,0.558501,0.048283,0.356086,0.6000000000000001,False
4,SOW_OTROSBCOS_ULT_RCC,"[0.6000000000000001,0.9)",556,0.312711,516,40,0.071942,0.172922,0.010086,0.356086,0.9,False
5,SOW_OTROSBCOS_ULT_RCC,"[0.9,1.0)",335,0.188414,321,14,0.041791,-0.402235,0.025605,0.356086,1.0,False
6,SOW_OTROSBCOS_ULT_RCC,"[1.0,inf)",213,0.119798,210,3,0.014085,-1.518346,0.149102,0.356086,inf,False


In [282]:
data = add_new_var_x_cut_up(data, colum='SOW_OTROSBCOS_ULT_RCC', cortes=[0.6, 0.7, 0.8, 0.9], impute=0, up=False, soft=True)

>>  _0.6_cut_down_SOW_OTROSBCOS_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4953271028037383  PRECISION:  0.05344897135941912  F1:  0.09648643728381576  GINI:  -0.039000575666032766
//////////////////////////////////////////////////
>>  _0.7_cut_down_SOW_OTROSBCOS_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5841121495327103  PRECISION:  0.05540780141843971  F1:  0.10121457489878542  GINI:  -0.022457359746579253
//////////////////////////////////////////////////
>>  _0.8_cut_down_SOW_OTROSBCOS_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6934579439252336  PRECISION:  0.05799140289175459  F1:  0.10703209520375044  GINI:  0.007302871626474694
//////////////////////////////////////////////////
>>  _0.9_cut_down_SOW_OTROSBCOS_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.8280373831775701  PRECISION:  0.061314878892733564  F1:  0.11417525773195876  GINI:  0.05586386615605132
/////////

In [92]:
var = vars_top[25]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN__IBKSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN__IBKSF_ULT_RCC,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.355107,missing,True
1,DEBT_IN__IBKSF_ULT_RCC,"[-inf,500000.0)",568,0.319460,545,23,0.040493,-0.435143,0.050104,0.355107,500000.0,False
2,DEBT_IN__IBKSF_ULT_RCC,"[500000.0,2000000.0)",219,0.123172,200,19,0.086758,0.376270,0.020586,0.355107,2000000.0,False
3,DEBT_IN__IBKSF_ULT_RCC,"[2000000.0,6000000.0)",343,0.192913,325,18,0.052478,-0.163305,0.004788,0.355107,6000000.0,False
4,DEBT_IN__IBKSF_ULT_RCC,"[6000000.0,8000000.0)",98,0.055118,96,2,0.020408,-1.141052,0.044637,0.355107,8000000.0,False
5,DEBT_IN__IBKSF_ULT_RCC,"[8000000.0,11000000.0)",214,0.120360,192,22,0.102804,0.563696,0.049070,0.355107,11000000.0,False
6,DEBT_IN__IBKSF_ULT_RCC,"[11000000.0,17500000.0)",136,0.076490,127,9,0.066176,0.083186,0.000549,0.355107,17500000.0,False
7,DEBT_IN__IBKSF_ULT_RCC,"[17500000.0,inf)",101,0.056805,86,15,0.148515,0.983852,0.084851,0.355107,inf,False


In [93]:
var = vars_top[26]
print(">>>>> ", var)
display(cortes[var])

>>>>>  RIESGO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,RIESGO,"[-inf,100000.0)",514,0.289089,499,15,0.029183,-0.766108,0.122511,0.355092,100000.0,False
1,RIESGO,"[100000.0,500000.0)",358,0.201350,339,19,0.053073,-0.143113,0.003874,0.355092,500000.0,False
2,RIESGO,"[500000.0,800000.0)",287,0.161417,265,22,0.076655,0.249760,0.011244,0.355092,800000.0,False
3,RIESGO,"[800000.0,1200000.0)",211,0.118673,204,7,0.033175,-0.633762,0.036340,0.355092,1200000.0,False
4,RIESGO,"[1200000.0,4400000.0)",311,0.174916,284,27,0.086817,0.385310,0.030802,0.355092,4400000.0,False
5,RIESGO,"[4400000.0,inf)",97,0.054556,79,18,0.185567,1.259372,0.150320,0.355092,inf,False


In [283]:
data = add_new_var_x_cut_up(data, colum='RIESGO', cortes=[1200000, 4400000], impute=0, up=True, soft=False)

>>  _1200000_cut_up_RIESGO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.13738317757009347  PRECISION:  0.07546201232032854  F1:  0.09741550695825049  GINI:  0.034855567414110444
//////////////////////////////////////////////////
>>  _4400000_cut_up_RIESGO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.05514018691588785  PRECISION:  0.07082833133253301  F1:  0.06200735680504466  GINI:  0.011077793656181445
//////////////////////////////////////////////////


In [94]:
data = add_new_var_x_cut_up(data, colum='COVER_GAR_AUTO_HIPO_ULT_RCC', cortes=[0.1, 0.2, 0.3], impute=0, up=False, soft=True)

>>  _0.1_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4289719626168224  PRECISION:  0.04517716535433071  F1:  0.0817453250222618  GINI:  -0.12328808520282908
//////////////////////////////////////////////////
>>  _0.2_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4934579439252336  PRECISION:  0.04806991988346686  F1:  0.08760577401692385  GINI:  -0.1017828621774648
//////////////////////////////////////////////////
>>  _0.3_cut_down_COVER_GAR_AUTO_HIPO_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5644859813084112  PRECISION:  0.05058202830583703  F1:  0.0928445161786181  GINI:  -0.0809085308172861
//////////////////////////////////////////////////


In [95]:
var = vars_top[27]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U12M_ACTUAL_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U12M_ACTUAL_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.352241,missing,True
1,ADEX_MAX_U12M_ACTUAL_IMP,"[-inf,100000.0)",474,0.266592,452,22,0.046414,-0.284192,0.019028,0.352241,100000.0,False
2,ADEX_MAX_U12M_ACTUAL_IMP,"[100000.0,400000.0)",278,0.156355,273,5,0.017986,-1.261586,0.147829,0.352241,400000.0,False
3,ADEX_MAX_U12M_ACTUAL_IMP,"[400000.0,700000.0)",103,0.057930,86,17,0.165049,1.117314,0.118335,0.352241,700000.0,False
4,ADEX_MAX_U12M_ACTUAL_IMP,"[700000.0,1700000.0)",278,0.156355,266,12,0.043165,-0.360142,0.017348,0.352241,1700000.0,False
5,ADEX_MAX_U12M_ACTUAL_IMP,"[1700000.0,2700000.0)",167,0.093926,150,17,0.101796,0.561026,0.037918,0.352241,2700000.0,False
6,ADEX_MAX_U12M_ACTUAL_IMP,"[2700000.0,inf)",190,0.106862,176,14,0.073684,0.207021,0.005018,0.352241,inf,False


In [96]:
var = vars_top[28]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TMO_DEUDA_TOTAL_FULL_RCC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TMO_DEUDA_TOTAL_FULL_RCC_SF,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.352096,missing,True
1,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[-inf,25000000.0)",403,0.226659,391,12,0.029777,-0.753652,0.093407,0.352096,25000000.0,False
2,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[25000000.0,105000000.0)",926,0.520810,884,42,0.045356,-0.316639,0.045490,0.352096,105000000.0,False
3,TMO_DEUDA_TOTAL_FULL_RCC_SF,"[105000000.0,inf)",448,0.251969,394,54,0.120536,0.742782,0.193078,0.352096,inf,False


In [97]:
data = add_new_var_x_cut_up(data, colum='RIESGO', cortes=[1100000.0], impute=0, up=True, soft=False)

>>  _1100000.0_cut_up_RIESGO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1485981308411215  PRECISION:  0.07662650602409639  F1:  0.10111287758346582  GINI:  0.039523782668515306
//////////////////////////////////////////////////


In [98]:
var = vars_top[29]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_IBK


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_IBK,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.34159,missing,True
1,GAR_TOT_IBK,"[-inf,1000000.0)",711,0.399888,689,22,0.030942,-0.705751,0.147411,0.34159,1000000.0,False
2,GAR_TOT_IBK,"[1000000.0,2000000.0)",98,0.055118,85,13,0.132653,0.860746,0.059798,0.34159,2000000.0,False
3,GAR_TOT_IBK,"[2000000.0,7000000.0)",340,0.191226,317,23,0.067647,0.115040,0.002662,0.34159,7000000.0,False
4,GAR_TOT_IBK,"[7000000.0,10000000.0)",138,0.077615,132,6,0.043478,-0.352595,0.008281,0.34159,10000000.0,False
5,GAR_TOT_IBK,"[10000000.0,inf)",374,0.210349,334,40,0.106952,0.616186,0.104980,0.34159,inf,False


In [284]:
data = add_new_var_x_cut_up(data, colum='GAR_TOT_IBK', cortes=[10000000], impute=0, up=True, soft=False)

>>  _10000000_cut_up_GAR_TOT_IBK
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1411214953271028  PRECISION:  0.08023379383634431  F1:  0.10230352303523037  GINI:  0.04257885613775958
//////////////////////////////////////////////////


In [99]:
# 22 000 000

In [100]:
var = vars_top[30]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U6M_ACTUAL_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U6M_ACTUAL_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.324657,missing,True
1,ADEX_MAX_U6M_ACTUAL_IMP,"[-inf,100000.0)",577,0.324522,548,29,0.050260,-0.200532,0.011957,0.324657,100000.0,False
2,ADEX_MAX_U6M_ACTUAL_IMP,"[100000.0,300000.0)",207,0.116423,202,5,0.024155,-0.960382,0.071704,0.324657,300000.0,False
3,ADEX_MAX_U6M_ACTUAL_IMP,"[300000.0,700000.0)",171,0.096175,150,21,0.122807,0.772335,0.080805,0.324657,700000.0,False
4,ADEX_MAX_U6M_ACTUAL_IMP,"[700000.0,1200000.0)",142,0.079865,140,2,0.014085,-1.510048,0.098627,0.324657,1200000.0,False
5,ADEX_MAX_U6M_ACTUAL_IMP,"[1200000.0,1500000.0)",95,0.053431,83,12,0.126316,0.804514,0.049406,0.324657,1500000.0,False
6,ADEX_MAX_U6M_ACTUAL_IMP,"[1500000.0,2800000.0)",174,0.097863,165,9,0.051724,-0.170273,0.002634,0.324657,2800000.0,False
7,ADEX_MAX_U6M_ACTUAL_IMP,"[2800000.0,inf)",124,0.069741,115,9,0.072581,0.190740,0.002760,0.324657,inf,False


In [101]:
[col for col in data.columns if 'TEXTIL' in col.upper()]

['MTO_ADEX_TEXTIL_EXP',
 'TEXTIL_EXP_PERCENT_ADEX',
 'MTO_ADEX_TEXTIL_IMP',
 'TEXTIL_IMP_PERCENT_ADEX',
 'TEXTIL_EXP_ADEX',
 'TEXTIL_IMP_ADEX',
 'SEI_MCDO__TEXTIL']

In [102]:
data = add_new_var_x_cut_up(data, colum='TEXTIL_EXP_PERCENT_ADEX', cortes=[0.1], impute=0, up=True, soft=False)

>>  _0.1_cut_up_TEXTIL_EXP_PERCENT_ADEX
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.005607476635514018  PRECISION:  0.05  F1:  0.010084033613445379  GINI:  -0.0008823332244426441
//////////////////////////////////////////////////


In [103]:
var = vars_top[31]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_PRESTAMOS_VIGENTE_U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_PRESTAMOS_VIGENTE_U2M,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.322153,missing,True
1,MONTO_PRESTAMOS_VIGENTE_U2M,"[-inf,5000000.0)",372,0.209224,353,19,0.051075,-0.191880,0.007081,0.322153,5000000.0,False
2,MONTO_PRESTAMOS_VIGENTE_U2M,"[5000000.0,13000000.0)",624,0.350956,601,23,0.036859,-0.532952,0.079238,0.322153,13000000.0,False
3,MONTO_PRESTAMOS_VIGENTE_U2M,"[13000000.0,21000000.0)",348,0.195726,324,24,0.068966,0.127459,0.003362,0.322153,21000000.0,False
4,MONTO_PRESTAMOS_VIGENTE_U2M,"[21000000.0,inf)",347,0.195163,305,42,0.121037,0.747507,0.151774,0.322153,inf,False


In [104]:
var = vars_top[32]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U12M_PASADO_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U12M_PASADO_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.320149,missing,True
1,ADEX_MAX_U12M_PASADO_IMP,"[-inf,100000.0)",460,0.258718,436,24,0.052174,-0.161141,0.006261,0.320149,100000.0,False
2,ADEX_MAX_U12M_PASADO_IMP,"[100000.0,200000.0)",155,0.087177,154,1,0.006452,-2.298505,0.190675,0.320149,200000.0,False
3,ADEX_MAX_U12M_PASADO_IMP,"[200000.0,500000.0)",170,0.095613,163,7,0.041176,-0.409392,0.013424,0.320149,500000.0,False
4,ADEX_MAX_U12M_PASADO_IMP,"[500000.0,1000000.0)",217,0.122047,197,20,0.092166,0.450976,0.030315,0.320149,1000000.0,False
5,ADEX_MAX_U12M_PASADO_IMP,"[1000000.0,1600000.0)",152,0.085489,146,6,0.039474,-0.453399,0.014450,0.320149,1600000.0,False
6,ADEX_MAX_U12M_PASADO_IMP,"[1600000.0,2900000.0)",191,0.107424,179,12,0.062827,0.035969,0.000141,0.320149,2900000.0,False
7,ADEX_MAX_U12M_PASADO_IMP,"[2900000.0,inf)",145,0.081552,128,17,0.117241,0.719631,0.058118,0.320149,inf,False


In [105]:
var = vars_top[33]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_SFSF_ULT_RCC_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.320064,missing,True
1,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,"[-inf,1.0)",1334,0.750281,1268,66,0.049475,-0.225393,0.034535,0.320064,1.0,False
2,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,"[1.0,5.0)",177,0.099550,163,14,0.079096,0.275456,0.008526,0.320064,5.0,False
3,MAX_DIAS_ATRASO_SFSF_ULT_RCC_x,"[5.0,inf)",173,0.097300,145,28,0.161850,1.085620,0.184870,0.320064,inf,False


In [106]:
var = vars_top[34]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.31836,missing,True
1,ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO,"[-inf,0.5)",344,0.193476,328,16,0.046512,-0.281977,0.013608,0.31836,0.5,False
2,ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO,"[0.5,0.6000000000000001)",89,0.050056,87,2,0.022472,-1.034313,0.034729,0.31836,0.6000000000000001,False
3,ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO,"[0.6000000000000001,0.9)",220,0.123735,194,26,0.118182,0.728686,0.090775,0.31836,0.9,False
4,ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO,"[0.9,1.5)",575,0.323397,536,39,0.067826,0.117875,0.004733,0.31836,1.5,False
5,ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO,"[1.5,inf)",262,0.147357,258,4,0.015267,-1.428218,0.167750,0.31836,inf,False


In [285]:
data = add_new_var_x_cut_up(data, colum='ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO', cortes=[0.5], impute=0, up=False, soft=True)

>>  _0.5_cut_down_ADEX_MAX_U6M_ACTUAL_IMP_OVER_12M_PASADO
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1308411214953271  PRECISION:  0.059931506849315065  F1:  0.08220786846741046  GINI:  0.005826889456160611
//////////////////////////////////////////////////


In [107]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN', cortes=[0. -19], impute=0, up=False, soft=True)

>>  _-19.0_cut_down_DIFF_PROM_CANT_TRX_U6M_U12M_CASHIN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04953271028037383  PRECISION:  0.11300639658848614  F1:  0.06887589343729694  GINI:  0.02585059710719828
//////////////////////////////////////////////////


In [108]:
var = vars_top[35]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF,"[-inf,3.0)",1670,0.939258,1587,83,0.049701,-0.212312,0.038594,0.318261,3.0,False
1,ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF,"[3.0,inf)",108,0.060742,83,25,0.231481,1.538483,0.279667,0.318261,inf,False


In [109]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF', cortes=[2, 5, 6], impute=0, up=True, soft=False)

>>  _2_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12990654205607477  PRECISION:  0.18508655126498003  F1:  0.1526633717737507  GINI:  0.09506651017630707
//////////////////////////////////////////////////
>>  _5_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08224299065420561  PRECISION:  0.2558139534883721  F1:  0.12446958981612449  GINI:  0.06766938254763621
//////////////////////////////////////////////////
>>  _6_cut_up_ATRASO_PROM_U12M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.06635514018691589  PRECISION:  0.2679245283018868  F1:  0.10636704119850188  GINI:  0.05531107779365607
//////////////////////////////////////////////////


In [110]:
var = vars_top[36]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.316066,missing,True
1,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[-inf,0.1)",262,0.147357,237,25,0.095420,0.480965,0.042158,0.316066,0.1,False
2,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[0.1,0.5)",256,0.143982,246,10,0.039062,-0.472598,0.026219,0.316066,0.5,False
3,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[0.5,0.6000000000000001)",120,0.067492,105,15,0.125000,0.784239,0.058713,0.316066,0.6000000000000001,False
4,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[0.6000000000000001,0.8)",268,0.150731,250,18,0.067164,0.099060,0.001544,0.316066,0.8,False
5,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[0.8,0.9)",190,0.106862,173,17,0.089474,0.410071,0.021534,0.316066,0.9,False
6,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[0.9,1.0)",295,0.165917,281,14,0.047458,-0.269148,0.010687,0.316066,1.0,False
7,PERCENT_DIRECTA_VIGVENC_LINMER_OTROS_BANCOS_PRIN_V,"[1.0,inf)",293,0.164792,284,9,0.030717,-0.721601,0.063079,0.316066,inf,False


In [111]:
var = vars_top[37]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.316066,missing,True
1,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[-inf,0.1)",262,0.147357,237,25,0.095420,0.480965,0.042158,0.316066,0.1,False
2,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[0.1,0.5)",256,0.143982,246,10,0.039062,-0.472598,0.026219,0.316066,0.5,False
3,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[0.5,0.6000000000000001)",120,0.067492,105,15,0.125000,0.784239,0.058713,0.316066,0.6000000000000001,False
4,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[0.6000000000000001,0.8)",268,0.150731,250,18,0.067164,0.099060,0.001544,0.316066,0.8,False
5,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[0.8,0.9)",190,0.106862,173,17,0.089474,0.410071,0.021534,0.316066,0.9,False
6,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[0.9,1.0)",295,0.165917,281,14,0.047458,-0.269148,0.010687,0.316066,1.0,False
7,DEBT_PERCENT_OTHER_BANKS_PRINSF_ULT_RCC,"[1.0,inf)",293,0.164792,284,9,0.030717,-0.721601,0.063079,0.316066,inf,False


In [112]:
#data = add_new_var_x_cut_up(data, colum='TENDENCIA_PERCENT_CLASIF_NO_NORMAL_SEI_U3M', cortes=[0.0039], impute=0, up=True, soft=False)

In [113]:
var = vars_top[38]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.314596,missing,True
1,GAR_TOT,"[-inf,5000000.0)",288,0.161980,280,8,0.027778,-0.816900,0.076454,0.314596,5000000.0,False
2,GAR_TOT,"[5000000.0,45000000.0)",856,0.481440,815,41,0.047897,-0.251168,0.027225,0.314596,45000000.0,False
3,GAR_TOT,"[45000000.0,60000000.0)",167,0.093926,151,16,0.095808,0.493757,0.028504,0.314596,60000000.0,False
4,GAR_TOT,"[60000000.0,75000000.0)",98,0.055118,93,5,0.051020,-0.184714,0.001735,0.314596,75000000.0,False
5,GAR_TOT,"[75000000.0,inf)",252,0.141732,218,34,0.134921,0.880313,0.162220,0.314596,inf,False


In [114]:
data = add_new_var_x_cut_up(data, colum='DEBT_rati_u6m_yoy', cortes=[1.5, 1.9], impute=0, up=True, soft=False)

>>  _1.5_cut_up_DEBT_rati_u6m_yoy
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.9738317757009346  PRECISION:  0.060151244010852624  F1:  0.1133039743380634  GINI:  0.04698445701711362
//////////////////////////////////////////////////
>>  _1.9_cut_up_DEBT_rati_u6m_yoy
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.9392523364485982  PRECISION:  0.060934942096647064  F1:  0.1144451403518761  GINI:  0.05754904600114297
//////////////////////////////////////////////////


In [115]:
var = vars_top[39]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CANTIDAD_CASHOUT_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CANTIDAD_CASHOUT_U12M,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.312972,missing,True
1,CANTIDAD_CASHOUT_U12M,"[-inf,5.0)",175,0.098425,172,3,0.017143,-1.310435,0.098566,0.312972,5.0,False
2,CANTIDAD_CASHOUT_U12M,"[5.0,25.0)",514,0.289089,487,27,0.052529,-0.153980,0.006408,0.312972,25.0,False
3,CANTIDAD_CASHOUT_U12M,"[25.0,35.0)",147,0.082677,143,4,0.027211,-0.838103,0.040725,0.312972,35.0,False
4,CANTIDAD_CASHOUT_U12M,"[35.0,55.0)",201,0.113048,177,24,0.119403,0.740352,0.086054,0.312972,55.0,False
5,CANTIDAD_CASHOUT_U12M,"[55.0,100.0)",251,0.141170,238,13,0.051793,-0.168874,0.003740,0.312972,100.0,False
6,CANTIDAD_CASHOUT_U12M,"[100.0,130.0)",100,0.056243,87,13,0.130000,0.837489,0.057179,0.312972,130.0,False
7,CANTIDAD_CASHOUT_U12M,"[130.0,inf)",299,0.168166,278,21,0.070234,0.155349,0.004346,0.312972,inf,False


In [116]:
data = add_new_var_x_cut_up(data, colum='MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF', cortes=[24000000.0, 46000000.0], impute=0, up=True, soft=False)

>>  _24000000.0_cut_up_MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.15700934579439252  PRECISION:  0.07288503253796096  F1:  0.09955555555555555  GINI:  0.03535387499853693
//////////////////////////////////////////////////
>>  _46000000.0_cut_up_MTO_DEUDA_TOTAL_COLDIRECT_SIN_REATCIVA_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08411214953271028  PRECISION:  0.07042253521126761  F1:  0.0766609880749574  GINI:  0.016481499413161016
//////////////////////////////////////////////////


In [117]:
var = vars_top[40]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_CASHIN_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_CASHIN_U12M,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.308617,missing,True
1,TENDENCIA_CASHIN_U12M,"[-inf,-180000.0)",128,0.071991,124,4,0.031250,-0.695540,0.025884,0.308617,-180000.0,False
2,TENDENCIA_CASHIN_U12M,"[-180000.0,-40000.0)",226,0.127109,207,19,0.084071,0.350168,0.018200,0.308617,-40000.0,False
3,TENDENCIA_CASHIN_U12M,"[-40000.0,-20000.0)",137,0.077053,118,19,0.138686,0.912202,0.096025,0.308617,-20000.0,False
4,TENDENCIA_CASHIN_U12M,"[-20000.0,20000.0)",592,0.332958,564,28,0.047297,-0.264402,0.020746,0.308617,20000.0,False
5,TENDENCIA_CASHIN_U12M,"[20000.0,50000.0)",141,0.079303,138,3,0.021277,-1.090194,0.059805,0.308617,50000.0,False
6,TENDENCIA_CASHIN_U12M,"[50000.0,340000.0)",319,0.179415,291,28,0.087774,0.397329,0.033776,0.308617,340000.0,False
7,TENDENCIA_CASHIN_U12M,"[340000.0,inf)",144,0.080990,140,4,0.027778,-0.816900,0.038227,0.308617,inf,False


In [118]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF', cortes=[1, 2, 3, 4], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1691588785046729  PRECISION:  0.14811783960720132  F1:  0.1579406631762653  GINI:  0.1098966674150681
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12990654205607477  PRECISION:  0.17866323907455012  F1:  0.15043290043290042  GINI:  0.09352944994631729
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09906542056074766  PRECISION:  0.21721311475409835  F1:  0.1360718870346598  GINI:  0.077318864714226
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0691588785046729  PRECISION:  0.2379421221864952  F1:  0.10716871832005793

In [119]:
var = vars_top[41]
print(">>>>> ", var)
display(cortes[var])

>>>>>  EBITDA_S_SERVDEUDA_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,EBITDA_S_SERVDEUDA_CEF_SITUACION,missing,482,0.271091,461,21,0.043568,-0.350428,0.028596,0.306081,missing,True
1,EBITDA_S_SERVDEUDA_CEF_SITUACION,"[-inf,0.0)",92,0.051744,79,13,0.141304,0.933949,0.068239,0.306081,0.0,False
2,EBITDA_S_SERVDEUDA_CEF_SITUACION,"[0.0,2.0)",824,0.463442,762,62,0.075243,0.229636,0.027048,0.306081,2.0,False
3,EBITDA_S_SERVDEUDA_CEF_SITUACION,"[2.0,4.0)",234,0.131609,223,11,0.047009,-0.270829,0.008580,0.306081,4.0,False
4,EBITDA_S_SERVDEUDA_CEF_SITUACION,"[4.0,inf)",146,0.082115,145,1,0.006849,-2.238286,0.173617,0.306081,inf,False


In [120]:
var = vars_top[42]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_BANCOS_PRIN_DEUDASF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_BANCOS_PRIN_DEUDASF_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.303145,missing,True
1,NRO_BANCOS_PRIN_DEUDASF_ULT_RCC,"[-inf,2.0)",301,0.169291,275,26,0.086379,0.371474,0.027519,0.303145,2.0,False
2,NRO_BANCOS_PRIN_DEUDASF_ULT_RCC,"[2.0,3.0)",382,0.214848,372,10,0.026178,-0.886160,0.116022,0.303145,3.0,False
3,NRO_BANCOS_PRIN_DEUDASF_ULT_RCC,"[3.0,4.0)",429,0.241282,407,22,0.051282,-0.187622,0.007822,0.303145,4.0,False
4,NRO_BANCOS_PRIN_DEUDASF_ULT_RCC,"[4.0,5.0)",411,0.231159,377,34,0.082725,0.324264,0.028037,0.303145,5.0,False
5,NRO_BANCOS_PRIN_DEUDASF_ULT_RCC,"[5.0,inf)",161,0.090551,145,16,0.099379,0.526004,0.031611,0.303145,inf,False


In [121]:
var = vars_top[43]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTDO_COL_DIRECTAS_SF_6UM


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTDO_COL_DIRECTAS_SF_6UM,missing,49,0.027559,49,0,0.000000,-1.267032,0.026705,0.300787,missing,True
1,MTDO_COL_DIRECTAS_SF_6UM,"[-inf,16000000.0)",854,0.480315,822,32,0.037471,-0.515856,0.102329,0.300787,16000000.0,False
2,MTDO_COL_DIRECTAS_SF_6UM,"[16000000.0,28000000.0)",391,0.219910,370,21,0.053708,-0.138832,0.003987,0.300787,28000000.0,False
3,MTDO_COL_DIRECTAS_SF_6UM,"[28000000.0,inf)",484,0.272216,429,55,0.113636,0.676025,0.167765,0.300787,inf,False


In [122]:
var = vars_top[44]
print(">>>>> ", var)
display(cortes[var])

>>>>>  VARIACION_PERCENT_GAR_IBK_U12M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,VARIACION_PERCENT_GAR_IBK_U12M,missing,761,0.428009,716,45,0.059133,-0.028570,0.000345,0.296932,missing,True
1,VARIACION_PERCENT_GAR_IBK_U12M,"[-inf,-0.1)",154,0.086614,153,1,0.006494,-2.291990,0.188763,0.296932,-0.1,False
2,VARIACION_PERCENT_GAR_IBK_U12M,"[-0.1,-0.020000000000000046)",108,0.060742,96,12,0.111111,0.659006,0.035340,0.296932,-0.020000000000000046,False
3,VARIACION_PERCENT_GAR_IBK_U12M,"[-0.020000000000000046,0.0799999999999999)",577,0.324522,532,45,0.077990,0.268467,0.026338,0.296932,0.0799999999999999,False
4,VARIACION_PERCENT_GAR_IBK_U12M,"[0.0799999999999999,inf)",178,0.100112,173,5,0.028090,-0.805406,0.046147,0.296932,inf,False


In [123]:
var = vars_top[45]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.292645,missing,True
1,NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC,"[-inf,1.0)",1561,0.877953,1475,86,0.055093,-0.111917,0.010466,0.292645,1.0,False
2,NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC,"[1.0,inf)",118,0.066367,96,22,0.186441,1.256843,0.181658,0.292645,inf,False


In [124]:
var = vars_top[46]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SEI_MCDO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SEI_MCDO,"MINERÍA%,%AUTOMOTRIZ VEHÍCULOS%,%TELECOMUNICACIONES%,%OTROS%,%CONSTRUCCIÓN",219,0.123172,215,4,0.018265,-1.245896,0.114255,0.29204,"MINERÍA%,%AUTOMOTRIZ VEHÍCULOS%,%TELECOMUNICACIONES%,%OTROS%,%CONSTRUCCIÓN",False
1,SEI_MCDO,"SERVICIOS%,%HIDROCARBUROS%,%TRANSPORTE",545,0.306524,523,22,0.040367,-0.430091,0.047082,0.29204,"SERVICIOS%,%HIDROCARBUROS%,%TRANSPORTE",False
2,SEI_MCDO,"INDUSTRIA%,%AGROPECUARIO%,%INMOBILIARIO",453,0.254781,423,30,0.066225,0.092273,0.002259,0.29204,"INDUSTRIA%,%AGROPECUARIO%,%INMOBILIARIO",False
3,SEI_MCDO,COMERCIO,435,0.244657,402,33,0.075862,0.238503,0.015464,0.29204,COMERCIO,False
4,SEI_MCDO,"TEXTIL%,%ENERGÍA%,%PESCA%,%RESTAURANTES%,%TURISMO Y HOSPEDAJE",126,0.070866,107,19,0.150794,1.010058,0.112979,0.29204,"TEXTIL%,%ENERGÍA%,%PESCA%,%RESTAURANTES%,%TURISMO Y HOSPEDAJE",False


In [125]:
var = vars_top[47]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,missing,169,0.095051,159,10,0.059172,-0.027871,0.000073,0.290938,missing,True
1,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[-inf,0.2)",102,0.057368,100,2,0.019608,-1.173575,0.048541,0.290938,0.2,False
2,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.2,0.4)",216,0.121485,204,12,0.055556,-0.094766,0.001047,0.290938,0.4,False
3,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.4,0.5)",110,0.061867,109,1,0.009091,-1.952900,0.109382,0.290938,0.5,False
4,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.5,0.6000000000000001)",147,0.082677,142,5,0.034014,-0.607941,0.023548,0.290938,0.6000000000000001,False
5,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[0.6000000000000001,1.4000000000000001)",751,0.422385,704,47,0.062583,0.031817,0.000434,0.290938,1.4000000000000001,False
6,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[1.4000000000000001,2.1)",117,0.065804,100,17,0.145299,0.966491,0.094259,0.290938,2.1,False
7,ENDEUDAMIENTO_PAS_VEN_CEF_SITUACION,"[2.1,inf)",166,0.093363,152,14,0.084337,0.353624,0.013654,0.290938,inf,False


In [126]:
var = vars_top[48]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ORDEN_ANUAL_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ORDEN_ANUAL_CEF_SITUACION,missing,169,0.095051,159,10,0.059172,-0.027871,0.000073,0.28778,missing,True
1,ORDEN_ANUAL_CEF_SITUACION,"[-inf,5.0)",388,0.218223,364,24,0.061856,0.019348,0.000082,0.28778,5.0,False
2,ORDEN_ANUAL_CEF_SITUACION,"[5.0,9.0)",378,0.212598,369,9,0.023810,-0.975124,0.134201,0.28778,9.0,False
3,ORDEN_ANUAL_CEF_SITUACION,"[9.0,10.0)",161,0.090551,154,7,0.043478,-0.352595,0.009661,0.28778,10.0,False
4,ORDEN_ANUAL_CEF_SITUACION,"[10.0,13.0)",492,0.276715,461,31,0.063008,0.039037,0.000429,0.28778,13.0,False
5,ORDEN_ANUAL_CEF_SITUACION,"[13.0,inf)",190,0.106862,163,27,0.142105,0.940534,0.143333,0.28778,inf,False


In [127]:
var = vars_top[49]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_BBVA_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_BBVA_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.287218,missing,True
1,MTOSALDO_BBVA_V,"[-inf,8500000.0)",1293,0.727222,1224,69,0.053364,-0.145624,0.014464,0.287218,8500000.0,False
2,MTOSALDO_BBVA_V,"[8500000.0,12000000.0)",126,0.070866,122,4,0.031746,-0.687578,0.024975,0.287218,12000000.0,False
3,MTOSALDO_BBVA_V,"[12000000.0,inf)",265,0.149044,230,35,0.132075,0.847418,0.155646,0.287218,inf,False


In [128]:
var = vars_top[50]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_PREN_NO_PREFER


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_PREN_NO_PREFER,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.286642,missing,True
1,GAR_PREN_NO_PREFER,"[-inf,4000000.0)",472,0.265467,454,18,0.038136,-0.489278,0.051467,0.286642,4000000.0,False
2,GAR_PREN_NO_PREFER,"[4000000.0,40000000.0)",864,0.485939,819,45,0.052083,-0.162974,0.012020,0.286642,40000000.0,False
3,GAR_PREN_NO_PREFER,"[40000000.0,88000000.0)",231,0.129921,208,23,0.099567,0.536404,0.047425,0.286642,88000000.0,False
4,GAR_PREN_NO_PREFER,"[88000000.0,inf)",94,0.052868,76,18,0.191489,1.298086,0.157273,0.286642,inf,False


In [129]:
data = add_new_var_x_cut_up(data, colum='maximo_percent_sow_others_u12m', cortes=[0.6], impute=0, up=True, soft=False)

>>  _0.6_cut_up_maximo_percent_sow_others_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.694392523364486  PRECISION:  0.06057394423609979  F1:  0.11142771445710858  GINI:  0.03840937409885936
//////////////////////////////////////////////////


In [130]:
var = vars_top[51]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_OTROSBCOS_MAX_historia


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_OTROSBCOS_MAX_historia,missing,13,0.007312,13,0,0.000000,0.059839,0.000029,0.284793,missing,True
1,SOW_OTROSBCOS_MAX_historia,"[-inf,0.1)",213,0.119798,190,23,0.107981,0.618619,0.060272,0.284793,0.1,False
2,SOW_OTROSBCOS_MAX_historia,"[0.1,0.5)",178,0.100112,170,8,0.044944,-0.326208,0.009243,0.284793,0.5,False
3,SOW_OTROSBCOS_MAX_historia,"[0.5,1.0)",1083,0.609111,1010,73,0.067405,0.102903,0.006745,0.284793,1.0,False
4,SOW_OTROSBCOS_MAX_historia,"[1.0,inf)",291,0.163667,287,4,0.013746,-1.543039,0.208504,0.284793,inf,False


In [131]:
var = vars_top[52]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_OTROS_BANCOS_PRIN


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_OTROS_BANCOS_PRIN,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.280942,missing,True
1,PERCENT_GAR_OTROS_BANCOS_PRIN,"[-inf,0.8)",733,0.412261,678,55,0.075034,0.226634,0.023405,0.280942,0.8,False
2,PERCENT_GAR_OTROS_BANCOS_PRIN,"[0.8,0.9)",225,0.126547,203,22,0.097778,0.516284,0.042411,0.280942,0.9,False
3,PERCENT_GAR_OTROS_BANCOS_PRIN,"[0.9,1.0)",556,0.312711,530,26,0.046763,-0.276333,0.021174,0.280942,1.0,False
4,PERCENT_GAR_OTROS_BANCOS_PRIN,"[1.0,inf)",147,0.082677,146,1,0.006803,-2.245159,0.175495,0.280942,inf,False


In [132]:
display(cortes['ASIA_IMP_PERCENT_ADEX'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ASIA_IMP_PERCENT_ADEX,missing,677,0.380765,640,37,0.054653,-0.112103,0.004556,0.121931,missing,True
1,ASIA_IMP_PERCENT_ADEX,"[-inf,0.1)",489,0.275028,463,26,0.053170,-0.141183,0.005154,0.121931,0.1,False
2,ASIA_IMP_PERCENT_ADEX,"[0.1,0.30000000000000004)",127,0.071429,113,14,0.110236,0.650117,0.040284,0.121931,0.30000000000000004,False
3,ASIA_IMP_PERCENT_ADEX,"[0.30000000000000004,0.6000000000000001)",177,0.099550,168,9,0.050847,-0.188292,0.003251,0.121931,0.6000000000000001,False
4,ASIA_IMP_PERCENT_ADEX,"[0.6000000000000001,0.9)",89,0.050056,87,2,0.022472,-1.034313,0.034729,0.121931,0.9,False
5,ASIA_IMP_PERCENT_ADEX,"[0.9,1.0)",94,0.052868,87,7,0.074468,0.218450,0.002778,0.121931,1.0,False
6,ASIA_IMP_PERCENT_ADEX,"[1.0,inf)",125,0.070304,112,13,0.104000,0.584898,0.031178,0.121931,inf,False


In [133]:
[col for col in data.columns if 'ASIA' in col]

['MTO_ADEX_ASIA_EXP',
 'ASIA_EXP_PERCENT_ADEX',
 'MTO_ADEX_ASIA_IMP',
 'ASIA_IMP_PERCENT_ADEX',
 'ASIA_EXP_ADEX',
 'ASIA_IMP_ADEX']

In [134]:
data = add_new_var_x_cut_up(data, colum='ASIA_IMP_PERCENT_ADEX', cortes=[0.7, 0.5, 0.4, 0.3, 0.2, 0.1], impute=0, up=True, soft=False)

>>  _0.7_cut_up_ASIA_IMP_PERCENT_ADEX
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12242990654205607  PRECISION:  0.06766528925619834  F1:  0.08715901530272788  GINI:  0.01967458375940767
//////////////////////////////////////////////////
>>  _0.5_cut_up_ASIA_IMP_PERCENT_ADEX
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.13551401869158877  PRECISION:  0.06225848003434951  F1:  0.08531921153280377  GINI:  0.01118292453241776
//////////////////////////////////////////////////
>>  _0.4_cut_up_ASIA_IMP_PERCENT_ADEX
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1485981308411215  PRECISION:  0.062182244818146264  F1:  0.08767576509511994  GINI:  0.01208441115536485
//////////////////////////////////////////////////
>>  _0.3_cut_up_ASIA_IMP_PERCENT_ADEX
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.15981308411214953  PRECISION:  0.06057385759829968  F1:  0.08784998715643463  GINI:  0.008839612633156024
//////////////////

In [135]:
var = vars_top[53]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_BCP_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_BCP_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.28053,missing,True
1,MTOSALDO_BCP_V,"[-inf,4000000.0)",971,0.546119,916,55,0.056643,-0.082534,0.003586,0.28053,4000000.0,False
2,MTOSALDO_BCP_V,"[4000000.0,7000000.0)",163,0.091676,150,13,0.079755,0.284463,0.008407,0.28053,7000000.0,False
3,MTOSALDO_BCP_V,"[7000000.0,10000000.0)",134,0.075366,132,2,0.014925,-1.459506,0.088558,0.28053,10000000.0,False
4,MTOSALDO_BCP_V,"[10000000.0,14000000.0)",107,0.060180,102,5,0.046729,-0.285386,0.004328,0.28053,14000000.0,False
5,MTOSALDO_BCP_V,"[14000000.0,inf)",309,0.173791,276,33,0.106796,0.606256,0.083518,0.28053,inf,False


In [136]:
var = vars_top[54]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ROTACION_CTASXPAGAR_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ROTACION_CTASXPAGAR_CEF_SITUACION,missing,220,0.123735,206,14,0.063636,0.049629,0.000311,0.278347,missing,True
1,ROTACION_CTASXPAGAR_CEF_SITUACION,"[-inf,15.0)",182,0.102362,177,5,0.027473,-0.828264,0.049441,0.278347,15.0,False
2,ROTACION_CTASXPAGAR_CEF_SITUACION,"[15.0,55.0)",521,0.293026,490,31,0.059501,-0.021971,0.000140,0.278347,55.0,False
3,ROTACION_CTASXPAGAR_CEF_SITUACION,"[55.0,85.0)",279,0.156918,271,8,0.028674,-0.784230,0.069170,0.278347,85.0,False
4,ROTACION_CTASXPAGAR_CEF_SITUACION,"[85.0,95.0)",105,0.059055,99,6,0.057143,-0.064913,0.000242,0.278347,95.0,False
5,ROTACION_CTASXPAGAR_CEF_SITUACION,"[95.0,110.0)",94,0.052868,78,16,0.170213,1.154328,0.117097,0.278347,110.0,False
6,ROTACION_CTASXPAGAR_CEF_SITUACION,"[110.0,220.0)",287,0.161417,270,17,0.059233,-0.026761,0.000114,0.278347,220.0,False
7,ROTACION_CTASXPAGAR_CEF_SITUACION,"[220.0,inf)",90,0.050619,79,11,0.122222,0.766895,0.041831,0.278347,inf,False


In [137]:
var = vars_top[55]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.276362,missing,True
1,NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC,"[-inf,1.0)",1544,0.868391,1459,85,0.055052,-0.112706,0.010495,0.276362,1.0,False
2,NRO_BANCOS_PRIN_CON_DIAS_ATRASOSF_ULT_RCC,"[1.0,inf)",135,0.075928,112,23,0.170370,1.147144,0.165346,0.276362,inf,False


In [138]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC', cortes=[1, 2], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1560747663551402  PRECISION:  0.2014475271411339  F1:  0.17588204318062142  GINI:  0.11838832664205823
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_A_IBKSF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.014953271028037384  PRECISION:  0.43243243243243246  F1:  0.028906955736224028  GINI:  0.013757779738045262
//////////////////////////////////////////////////


In [139]:
var = vars_top[56]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCETN_DEBT_IN__BCPSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCETN_DEBT_IN__BCPSF_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.275714,missing,True
1,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[-inf,0.1)",803,0.451631,755,48,0.059776,-0.025368,0.000287,0.275714,0.1,False
2,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[0.1,0.2)",179,0.100675,159,20,0.111732,0.656977,0.058106,0.275714,0.2,False
3,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[0.2,0.30000000000000004)",132,0.074241,129,3,0.022727,-1.031051,0.051240,0.275714,0.30000000000000004,False
4,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[0.30000000000000004,0.4)",129,0.072553,122,7,0.054264,-0.127962,0.001123,0.275714,0.4,False
5,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[0.4,0.6000000000000001)",189,0.106299,175,14,0.074074,0.204420,0.004859,0.275714,0.6000000000000001,False
6,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[0.6000000000000001,0.8)",111,0.062430,99,12,0.108108,0.619936,0.031562,0.275714,0.8,False
7,PERCETN_DEBT_IN__BCPSF_ULT_RCC,"[0.8,inf)",141,0.079303,137,4,0.028369,-0.803538,0.036404,0.275714,inf,False


In [140]:
var = vars_top[57]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_LEASING_VIGENTE_U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_LEASING_VIGENTE_U2M,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.275619,missing,True
1,MONTO_LEASING_VIGENTE_U2M,"[-inf,200000.0)",823,0.462880,761,62,0.075334,0.222650,0.025302,0.275619,200000.0,False
2,MONTO_LEASING_VIGENTE_U2M,"[200000.0,1400000.0)",342,0.192351,334,8,0.023392,-1.001551,0.126734,0.275619,1400000.0,False
3,MONTO_LEASING_VIGENTE_U2M,"[1400000.0,5400000.0)",247,0.138920,234,13,0.052632,-0.160223,0.003324,0.275619,5400000.0,False
4,MONTO_LEASING_VIGENTE_U2M,"[5400000.0,9800000.0)",102,0.057368,95,7,0.068627,0.122182,0.000903,0.275619,9800000.0,False
5,MONTO_LEASING_VIGENTE_U2M,"[9800000.0,inf)",177,0.099550,159,18,0.101695,0.551616,0.038657,0.275619,inf,False


In [141]:
var = vars_top[58]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CANTIDAD_CASHOUT_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CANTIDAD_CASHOUT_U1M,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.275259,missing,True
1,CANTIDAD_CASHOUT_U1M,"[-inf,10.0)",304,0.170979,286,18,0.059211,-0.027172,0.000125,0.275259,10.0,False
2,CANTIDAD_CASHOUT_U1M,"[10.0,15.0)",148,0.083240,145,3,0.020270,-1.139674,0.067296,0.275259,15.0,False
3,CANTIDAD_CASHOUT_U1M,"[15.0,30.0)",282,0.158605,271,11,0.039007,-0.465776,0.028144,0.275259,30.0,False
4,CANTIDAD_CASHOUT_U1M,"[30.0,110.0)",604,0.339708,555,49,0.081126,0.311300,0.037782,0.275259,110.0,False
5,CANTIDAD_CASHOUT_U1M,"[110.0,155.0)",133,0.074803,129,4,0.030075,-0.735070,0.029556,0.275259,155.0,False
6,CANTIDAD_CASHOUT_U1M,"[155.0,280.0)",126,0.070866,109,17,0.134921,0.880313,0.081110,0.275259,280.0,False
7,CANTIDAD_CASHOUT_U1M,"[280.0,inf)",90,0.050619,87,3,0.033333,-0.628848,0.015292,0.275259,inf,False


In [142]:
var = vars_top[59]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CANT_TRX_CASHOUT_MES_ACTUAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CANT_TRX_CASHOUT_MES_ACTUAL,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.275259,missing,True
1,CANT_TRX_CASHOUT_MES_ACTUAL,"[-inf,10.0)",304,0.170979,286,18,0.059211,-0.027172,0.000125,0.275259,10.0,False
2,CANT_TRX_CASHOUT_MES_ACTUAL,"[10.0,15.0)",148,0.083240,145,3,0.020270,-1.139674,0.067296,0.275259,15.0,False
3,CANT_TRX_CASHOUT_MES_ACTUAL,"[15.0,30.0)",282,0.158605,271,11,0.039007,-0.465776,0.028144,0.275259,30.0,False
4,CANT_TRX_CASHOUT_MES_ACTUAL,"[30.0,110.0)",604,0.339708,555,49,0.081126,0.311300,0.037782,0.275259,110.0,False
5,CANT_TRX_CASHOUT_MES_ACTUAL,"[110.0,155.0)",133,0.074803,129,4,0.030075,-0.735070,0.029556,0.275259,155.0,False
6,CANT_TRX_CASHOUT_MES_ACTUAL,"[155.0,280.0)",126,0.070866,109,17,0.134921,0.880313,0.081110,0.275259,280.0,False
7,CANT_TRX_CASHOUT_MES_ACTUAL,"[280.0,inf)",90,0.050619,87,3,0.033333,-0.628848,0.015292,0.275259,inf,False


In [143]:
var = vars_top[60]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.274751,missing,True
1,NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC,"[-inf,2.0)",419,0.235658,391,28,0.066826,0.093646,0.002152,0.274751,2.0,False
2,NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC,"[2.0,3.0)",508,0.285714,485,23,0.045276,-0.318506,0.025231,0.274751,3.0,False
3,NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC,"[3.0,4.0)",488,0.274466,466,22,0.045082,-0.322994,0.024878,0.274751,4.0,False
4,NRO_PROD_DEUDA_VIGENTE_SF_ULT_RCC,"[4.0,inf)",276,0.155231,241,35,0.126812,0.800700,0.141791,0.274751,inf,False


In [144]:
var = vars_top[61]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO,missing,288,0.161980,267,21,0.072917,0.195721,6.764957e-03,0.272892,missing,True
1,ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[-inf,0.5)",255,0.143420,248,7,0.027451,-0.829071,6.938344e-02,0.272892,0.5,False
2,ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[0.5,1.2000000000000002)",792,0.445444,727,65,0.082071,0.323908,5.393805e-02,0.272892,1.2000000000000002,False
3,ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[1.2000000000000002,1.4000000000000001)",115,0.064679,108,7,0.060870,0.002227,3.209781e-07,0.272892,1.4000000000000001,False
4,ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[1.4000000000000001,3.4000000000000004)",239,0.134421,235,4,0.016736,-1.334843,1.383986e-01,0.272892,3.4000000000000004,False
5,ADEX_MAX_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[3.4000000000000004,inf)",89,0.050056,85,4,0.044944,-0.317909,4.406593e-03,0.272892,inf,False


In [145]:
var = vars_top[62]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ATRASO_PROM_U3M_COLDIRECT_VIG_VENC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ATRASO_PROM_U3M_COLDIRECT_VIG_VENC_SF,"[-inf,1.0)",1682,0.946007,1596,86,0.051130,-0.182461,0.029083,0.272224,1.0,False
1,ATRASO_PROM_U3M_COLDIRECT_VIG_VENC_SF,"[1.0,inf)",96,0.053993,74,22,0.229167,1.525425,0.243141,0.272224,inf,False


In [146]:
var = vars_top[63]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN__BBVASF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN__BBVASF_ULT_RCC,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.271704,missing,True
1,DEBT_IN__BBVASF_ULT_RCC,"[-inf,7500000.0)",1277,0.718223,1211,66,0.051684,-0.179398,0.021364,0.271704,7500000.0,False
2,DEBT_IN__BBVASF_ULT_RCC,"[7500000.0,12000000.0)",175,0.098425,162,13,0.074286,0.207502,0.004642,0.271704,12000000.0,False
3,DEBT_IN__BBVASF_ULT_RCC,"[12000000.0,18500000.0)",113,0.063555,94,19,0.168142,1.131293,0.133701,0.271704,18500000.0,False
4,DEBT_IN__BBVASF_ULT_RCC,"[18500000.0,inf)",114,0.064117,104,10,0.087719,0.388343,0.011476,0.271704,inf,False


In [147]:
var = vars_top[64]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_BBVA


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_BBVA,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.269417,missing,True
1,PERCENT_GAR_BBVA,"[-inf,0.1)",944,0.530934,887,57,0.060381,-0.006346,0.000021,0.269417,0.1,False
2,PERCENT_GAR_BBVA,"[0.1,0.30000000000000004)",301,0.169291,269,32,0.106312,0.609472,0.082412,0.269417,0.30000000000000004,False
3,PERCENT_GAR_BBVA,"[0.30000000000000004,0.5)",141,0.079303,131,10,0.070922,0.165835,0.002346,0.269417,0.5,False
4,PERCENT_GAR_BBVA,"[0.5,0.8)",165,0.092801,161,4,0.024242,-0.956662,0.056797,0.269417,0.8,False
5,PERCENT_GAR_BBVA,"[0.8,inf)",110,0.061867,109,1,0.009091,-1.952900,0.109382,0.269417,inf,False


In [148]:
var = vars_top[65]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.267429,missing,True
1,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[-inf,0.5)",269,0.151294,262,7,0.026022,-0.883987,0.081390,0.267429,0.5,False
2,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[0.5,1.2000000000000002)",920,0.517435,847,73,0.079348,0.287206,0.048463,0.267429,1.2000000000000002,False
3,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[1.2000000000000002,2.2)",188,0.105737,185,3,0.015957,-1.383296,0.114814,0.267429,2.2,False
4,ADEX_PROM_U12M_ACTUAL_IMP_OVER_12M_PASADO,"[2.2,inf)",113,0.063555,109,4,0.035398,-0.566606,0.015997,0.267429,inf,False


In [149]:
var = vars_top[66]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF,"[-inf,2.0)",1643,0.924072,1561,82,0.049909,-0.207915,0.036483,0.265508,2.0,False
1,NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF,"[2.0,inf)",135,0.075928,109,26,0.192593,1.305196,0.229024,0.265508,inf,False


In [150]:
var = vars_top[67]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT,missing,87,0.048931,83,4,0.045977,-0.294099,0.003724,0.264156,missing,True
1,TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT,"[-inf,-13.0)",149,0.083802,130,19,0.127517,0.815352,0.079971,0.264156,-13.0,False
2,TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT,"[-13.0,-4.0)",171,0.096175,159,12,0.070175,0.154450,0.002456,0.264156,-4.0,False
3,TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT,"[-4.0,-2.0)",126,0.070866,125,1,0.007937,-2.089866,0.137076,0.264156,-2.0,False
4,TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT,"[-2.0,-1.0)",95,0.053431,93,2,0.021053,-1.101005,0.040924,0.264156,-1.0,False
5,TENDENCIA_COMPORTAMIENTO_NRO_TRABAJADORES_SUNAT,"[-1.0,inf)",1150,0.646794,1080,70,0.060870,0.002227,0.000003,0.264156,inf,False


In [151]:
var = vars_top[68]
print(">>>>> ", var)
display(cortes[var])

>>>>>  minimo_percent_col_direct_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,minimo_percent_col_direct_u12m,missing,18,0.010124,18,0,0.000000,-0.265583,0.000668,0.263772,missing,True
1,minimo_percent_col_direct_u12m,"[-inf,0.1)",219,0.123172,209,10,0.045662,-0.309600,0.010317,0.263772,0.1,False
2,minimo_percent_col_direct_u12m,"[0.1,0.5)",169,0.095051,168,1,0.005917,-2.393815,0.218833,0.263772,0.5,False
3,minimo_percent_col_direct_u12m,"[0.5,0.8)",245,0.137795,232,13,0.053061,-0.151639,0.002964,0.263772,0.8,False
4,minimo_percent_col_direct_u12m,"[0.8,inf)",1127,0.633858,1043,84,0.074534,0.211109,0.030991,0.263772,inf,False


In [152]:
var = vars_top[69]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_PROM_U12M_PASADO_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_PROM_U12M_PASADO_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.263567,missing,True
1,ADEX_PROM_U12M_PASADO_IMP,"[-inf,50000.0)",587,0.330146,559,28,0.047700,-0.255497,0.019283,0.263567,50000.0,False
2,ADEX_PROM_U12M_PASADO_IMP,"[50000.0,100000.0)",123,0.069179,121,2,0.016260,-1.364196,0.073580,0.263567,100000.0,False
3,ADEX_PROM_U12M_PASADO_IMP,"[100000.0,200000.0)",92,0.051744,89,3,0.032609,-0.651576,0.016625,0.263567,200000.0,False
4,ADEX_PROM_U12M_PASADO_IMP,"[200000.0,650000.0)",271,0.152418,246,25,0.092251,0.451992,0.038047,0.263567,650000.0,False
5,ADEX_PROM_U12M_PASADO_IMP,"[650000.0,900000.0)",94,0.052868,92,2,0.021277,-1.090194,0.039870,0.263567,900000.0,False
6,ADEX_PROM_U12M_PASADO_IMP,"[900000.0,1900000.0)",221,0.124297,208,13,0.058824,-0.034141,0.000143,0.263567,1900000.0,False
7,ADEX_PROM_U12M_PASADO_IMP,"[1900000.0,inf)",102,0.057368,88,14,0.137255,0.900168,0.069254,0.263567,inf,False


In [153]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_CON_ATRASO_U6M_COLDIRECT_SF', cortes=[2, 3], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.12990654205607477  PRECISION:  0.17866323907455012  F1:  0.15043290043290042  GINI:  0.09352944994631729
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_MESES_CON_ATRASO_U6M_COLDIRECT_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09906542056074766  PRECISION:  0.21721311475409835  F1:  0.1360718870346598  GINI:  0.077318864714226
//////////////////////////////////////////////////


In [154]:
var = vars_top[70]
print(">>>>> ", var)
display(cortes[var])

>>>>>  RIESGO_INDIRECTO_CIERRE_MES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,RIESGO_INDIRECTO_CIERRE_MES,missing,131,0.073678,130,1,0.007634,-2.129087,0.146024,0.257559,missing,True
1,RIESGO_INDIRECTO_CIERRE_MES,"[-inf,25000.0)",1174,0.660292,1091,83,0.070698,0.162438,0.018717,0.257559,25000.0,False
2,RIESGO_INDIRECTO_CIERRE_MES,"[25000.0,75000.0)",99,0.055681,91,8,0.080808,0.307030,0.006013,0.257559,75000.0,False
3,RIESGO_INDIRECTO_CIERRE_MES,"[75000.0,245000.0)",133,0.074803,131,2,0.015038,-1.443602,0.086507,0.257559,245000.0,False
4,RIESGO_INDIRECTO_CIERRE_MES,"[245000.0,inf)",241,0.135546,227,14,0.058091,-0.047445,0.000299,0.257559,inf,False


In [155]:
var = vars_top[71]
print(">>>>> ", var)
display(cortes[var])

>>>>>  maximo_auto_hipo_percent_pren_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,maximo_auto_hipo_percent_pren_u12m,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.256507,missing,True
1,maximo_auto_hipo_percent_pren_u12m,"[-inf,0.1)",400,0.224972,373,27,0.067500,0.112706,0.003003,0.256507,0.1,False
2,maximo_auto_hipo_percent_pren_u12m,"[0.1,0.2)",119,0.066929,115,4,0.033613,-0.620190,0.019738,0.256507,0.2,False
3,maximo_auto_hipo_percent_pren_u12m,"[0.2,0.30000000000000004)",124,0.069741,105,19,0.153226,1.028926,0.116322,0.256507,0.30000000000000004,False
4,maximo_auto_hipo_percent_pren_u12m,"[0.30000000000000004,0.4)",153,0.086052,147,6,0.039216,-0.460225,0.014943,0.256507,0.4,False
5,maximo_auto_hipo_percent_pren_u12m,"[0.4,0.5)",134,0.075366,120,14,0.104478,0.590013,0.034087,0.256507,0.5,False
6,maximo_auto_hipo_percent_pren_u12m,"[0.5,0.7000000000000001)",271,0.152418,262,9,0.033210,-0.632672,0.046535,0.256507,0.7000000000000001,False
7,maximo_auto_hipo_percent_pren_u12m,"[0.7000000000000001,inf)",460,0.258718,435,25,0.054348,-0.118023,0.003422,0.256507,inf,False


In [156]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_4SF_ULT_RCC', cortes=[1, 2], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_4SF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03457943925233645  PRECISION:  0.2569444444444444  F1:  0.06095551894563427  GINI:  0.028488126489043752
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_PRIN_CON_DIAS_ATRASO_MAYOR_4SF_ULT_RCC
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0009345794392523365  PRECISION:  0.5  F1:  0.0018656716417910447  GINI:  0.0008776512825861005
//////////////////////////////////////////////////


In [157]:
var = vars_top[72]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_SCOTIA


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_SCOTIA,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.255895,missing,True
1,GAR_TOT_SCOTIA,"[-inf,1000000.0)",978,0.550056,915,63,0.064417,0.062658,0.002220,0.255895,1000000.0,False
2,GAR_TOT_SCOTIA,"[1000000.0,5000000.0)",144,0.080990,142,2,0.013889,-1.524232,0.101379,0.255895,5000000.0,False
3,GAR_TOT_SCOTIA,"[5000000.0,9000000.0)",106,0.059618,102,4,0.037736,-0.500231,0.012026,0.255895,9000000.0,False
4,GAR_TOT_SCOTIA,"[9000000.0,15000000.0)",111,0.062430,108,3,0.027027,-0.845071,0.031177,0.255895,15000000.0,False
5,GAR_TOT_SCOTIA,"[15000000.0,34000000.0)",178,0.100112,165,13,0.073034,0.197452,0.004259,0.255895,34000000.0,False
6,GAR_TOT_SCOTIA,"[34000000.0,inf)",144,0.080990,125,19,0.131944,0.854573,0.086376,0.255895,inf,False


In [158]:
var = vars_top[73]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_BIF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_BIF,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.255231,missing,True
1,GAR_TOT_BIF,"[-inf,1600000.0)",1146,0.644544,1082,64,0.055846,-0.089236,0.004936,0.255231,1600000.0,False
2,GAR_TOT_BIF,"[1600000.0,2400000.0)",97,0.054556,88,9,0.092784,0.458335,0.014043,0.255231,2400000.0,False
3,GAR_TOT_BIF,"[2400000.0,8400000.0)",227,0.127672,222,5,0.022026,-1.054792,0.091385,0.255231,8400000.0,False
4,GAR_TOT_BIF,"[8400000.0,inf)",191,0.107424,165,26,0.136126,0.890599,0.126410,0.255231,inf,False


In [159]:
data = add_new_var_x_cut_up(data, colum='DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT', cortes=[-34, -30], impute=0, up=True, soft=False)

>>  _-34_cut_up_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.9196261682242991  PRECISION:  0.05593133632694822  F1:  0.10544928468091946  GINI:  -0.02589358584606405
//////////////////////////////////////////////////
>>  _-30_cut_up_DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.9149532710280374  PRECISION:  0.05592687803484719  F1:  0.10541049798115747  GINI:  -0.02584144603902394
//////////////////////////////////////////////////


In [160]:
var = vars_top[74]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.25471,missing,True
1,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[-inf,200.0)",310,0.174353,298,12,0.038710,-0.473739,0.031898,0.25471,200.0,False
2,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[200.0,300.0)",203,0.114173,187,16,0.078818,0.279928,0.010126,0.25471,300.0,False
3,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[300.0,400.0)",191,0.107424,186,5,0.026178,-0.877861,0.057132,0.25471,400.0,False
4,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[400.0,700.0)",340,0.191226,314,26,0.076471,0.247151,0.013029,0.25471,700.0,False
5,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[700.0,1100.0)",156,0.087739,146,10,0.064103,0.057426,0.000297,0.25471,1100.0,False
6,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[1100.0,2100.0)",122,0.068616,120,2,0.016393,-1.355897,0.072321,0.25471,2100.0,False
7,MTO_CASHOUT_OTROS_NO_TABLERO_MES_PAS,"[2100.0,inf)",365,0.205287,331,34,0.093151,0.462690,0.053955,0.25471,inf,False


In [161]:
var = vars_top[75]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.25471,missing,True
1,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[-inf,200.0)",310,0.174353,298,12,0.038710,-0.473739,0.031898,0.25471,200.0,False
2,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[200.0,300.0)",203,0.114173,187,16,0.078818,0.279928,0.010126,0.25471,300.0,False
3,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[300.0,400.0)",191,0.107424,186,5,0.026178,-0.877861,0.057132,0.25471,400.0,False
4,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[400.0,700.0)",340,0.191226,314,26,0.076471,0.247151,0.013029,0.25471,700.0,False
5,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[700.0,1100.0)",156,0.087739,146,10,0.064103,0.057426,0.000297,0.25471,1100.0,False
6,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[1100.0,2100.0)",122,0.068616,120,2,0.016393,-1.355897,0.072321,0.25471,2100.0,False
7,TRX_CASHOUT_OTROS_NO_TABLERO_2UM_MASIVO,"[2100.0,inf)",365,0.205287,331,34,0.093151,0.462690,0.053955,0.25471,inf,False


In [162]:
data = add_new_var_x_cut_up(data, colum='PERCENT_COL_DIRECTAS_SF_MAX_historia', cortes=[0.8, 0.9], impute=0, up=True, soft=False)

>>  _0.8_cut_up_PERCENT_COL_DIRECTAS_SF_MAX_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.8504672897196262  PRECISION:  0.0590295796574987  F1:  0.1103967002304986  GINI:  0.024667449118464857
//////////////////////////////////////////////////
>>  _0.9_cut_up_PERCENT_COL_DIRECTAS_SF_MAX_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7523364485981309  PRECISION:  0.057735064189916085  F1:  0.10724039166056086  GINI:  0.004414326316450445
//////////////////////////////////////////////////


In [163]:
var = vars_top[76]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_ENTIDADES_FULL_RCC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_ENTIDADES_FULL_RCC_SF,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.250246,missing,True
1,NRO_ENTIDADES_FULL_RCC_SF,"[-inf,2.0)",170,0.095613,157,13,0.076471,0.238852,0.006058,0.250246,2.0,False
2,NRO_ENTIDADES_FULL_RCC_SF,"[2.0,3.0)",270,0.151856,259,11,0.040741,-0.428784,0.023189,0.250246,3.0,False
3,NRO_ENTIDADES_FULL_RCC_SF,"[3.0,4.0)",364,0.204724,353,11,0.030220,-0.738424,0.081498,0.250246,4.0,False
4,NRO_ENTIDADES_FULL_RCC_SF,"[4.0,6.0)",650,0.365579,614,36,0.055385,-0.106327,0.003943,0.250246,6.0,False
5,NRO_ENTIDADES_FULL_RCC_SF,"[6.0,inf)",323,0.181665,286,37,0.114551,0.685075,0.115438,0.250246,inf,False


In [164]:
var = vars_top[77]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CAJA_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CAJA_CEF_SITUACION,missing,170,0.095613,160,10,0.058824,-0.034141,0.000110,0.24732,missing,True
1,CAJA_CEF_SITUACION,"[-inf,1500.0)",408,0.229471,378,30,0.073529,0.204751,0.010530,0.24732,1500.0,False
2,CAJA_CEF_SITUACION,"[1500.0,2000.0)",93,0.052306,81,12,0.129032,0.828905,0.051896,0.24732,2000.0,False
3,CAJA_CEF_SITUACION,"[2000.0,4500.0)",365,0.205287,349,16,0.043836,-0.344036,0.020929,0.24732,4500.0,False
4,CAJA_CEF_SITUACION,"[4500.0,6000.0)",139,0.078178,127,12,0.086331,0.379167,0.013295,0.24732,6000.0,False
5,CAJA_CEF_SITUACION,"[6000.0,7000.0)",100,0.056243,99,1,0.010000,-1.856672,0.092875,0.24732,7000.0,False
6,CAJA_CEF_SITUACION,"[7000.0,10000.0)",148,0.083240,143,5,0.033784,-0.614959,0.024188,0.24732,10000.0,False
7,CAJA_CEF_SITUACION,"[10000.0,12500.0)",97,0.054556,87,10,0.103093,0.575125,0.023291,0.24732,12500.0,False
8,CAJA_CEF_SITUACION,"[12500.0,inf)",258,0.145107,246,12,0.046512,-0.281977,0.010206,0.24732,inf,False


In [165]:
var = vars_top[78]
print(">>>>> ", var)
display(cortes[var])

>>>>>  EBITDA_ANUAL_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,EBITDA_ANUAL_CEF_SITUACION,missing,169,0.095051,159,10,0.059172,-0.027871,0.000073,0.246512,missing,True
1,EBITDA_ANUAL_CEF_SITUACION,"[-inf,0.0)",153,0.086052,137,16,0.104575,0.591055,0.039076,0.246512,0.0,False
2,EBITDA_ANUAL_CEF_SITUACION,"[0.0,1000.0)",170,0.095613,160,10,0.058824,-0.034141,0.000110,0.246512,1000.0,False
3,EBITDA_ANUAL_CEF_SITUACION,"[1000.0,3000.0)",296,0.166479,290,6,0.020270,-1.139674,0.134592,0.246512,3000.0,False
4,EBITDA_ANUAL_CEF_SITUACION,"[3000.0,8000.0)",427,0.240157,393,34,0.079625,0.290999,0.023130,0.246512,8000.0,False
5,EBITDA_ANUAL_CEF_SITUACION,"[8000.0,10000.0)",115,0.064679,112,3,0.026087,-0.881439,0.034630,0.246512,10000.0,False
6,EBITDA_ANUAL_CEF_SITUACION,"[10000.0,22000.0)",323,0.181665,305,18,0.055728,-0.091492,0.001461,0.246512,22000.0,False
7,EBITDA_ANUAL_CEF_SITUACION,"[22000.0,inf)",125,0.070304,114,11,0.088000,0.400145,0.013440,0.246512,inf,False


In [166]:
var = vars_top[79]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CANTIDAD_CASHOUT_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CANTIDAD_CASHOUT_U6M,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.245884,missing,True
1,CANTIDAD_CASHOUT_U6M,"[-inf,5.0)",145,0.081552,141,4,0.027586,-0.824018,0.039054,0.245884,5.0,False
2,CANTIDAD_CASHOUT_U6M,"[5.0,45.0)",854,0.480315,806,48,0.056206,-0.082435,0.003148,0.245884,45.0,False
3,CANTIDAD_CASHOUT_U6M,"[45.0,55.0)",90,0.050619,78,12,0.133333,0.866646,0.055816,0.245884,55.0,False
4,CANTIDAD_CASHOUT_U6M,"[55.0,220.0)",493,0.277278,453,40,0.081136,0.311435,0.030867,0.245884,220.0,False
5,CANTIDAD_CASHOUT_U6M,"[220.0,inf)",105,0.059055,104,1,0.009524,-1.905943,0.101046,0.245884,inf,False


In [167]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M', cortes=[1, 2, 3, 4], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.5906542056074766  PRECISION:  0.06431915326684307  F1:  0.11600587371512483  GINI:  0.06725673321763259
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.49439252336448597  PRECISION:  0.06553518334985134  F1:  0.11572959964996717  GINI:  0.06498343763068193
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.29345794392523367  PRECISION:  0.07203487038311539  F1:  0.11567507828329342  GINI:  0.06318355021010214
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1766355140186916  PRECISION:  0.06860254083484574

In [168]:
var = vars_top[80]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,missing,252,0.141732,233,19,0.075397,0.231848,0.008440,0.242391,missing,True
1,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,"[-inf,3.0)",860,0.483690,819,41,0.047674,-0.256064,0.028369,0.242391,3.0,False
2,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,"[3.0,4.0)",416,0.233971,401,15,0.036058,-0.547464,0.055420,0.242391,4.0,False
3,NRO_PROD_OTRA_ENTIDAD_DEUDA_VIGENTE_SF_ULT_RCC,"[4.0,inf)",250,0.140607,217,33,0.132000,0.855058,0.150161,0.242391,inf,False


In [169]:
var = vars_top[81]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_PROM_U12M_ACTUAL_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_PROM_U12M_ACTUAL_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.242328,missing,True
1,ADEX_PROM_U12M_ACTUAL_IMP,"[-inf,50000.0)",597,0.335771,570,27,0.045226,-0.311352,0.028432,0.242328,50000.0,False
2,ADEX_PROM_U12M_ACTUAL_IMP,"[50000.0,150000.0)",187,0.105174,184,3,0.016043,-1.377876,0.113540,0.242328,150000.0,False
3,ADEX_PROM_U12M_ACTUAL_IMP,"[150000.0,300000.0)",101,0.056805,87,14,0.138614,0.911597,0.070680,0.242328,300000.0,False
4,ADEX_PROM_U12M_ACTUAL_IMP,"[300000.0,900000.0)",305,0.171541,287,18,0.059016,-0.030663,0.000159,0.242328,900000.0,False
5,ADEX_PROM_U12M_ACTUAL_IMP,"[900000.0,inf)",300,0.168729,275,25,0.083333,0.340552,0.022753,0.242328,inf,False


In [170]:
data = add_new_var_x_cut_up(data, colum='F_Septile', cortes=[2, 3], impute=0, up=False, soft=True)

>>  _2_cut_down_F_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.08037383177570094  PRECISION:  0.0529882932840419  F1:  0.06386929075380617  GINI:  -0.007124745020382295
//////////////////////////////////////////////////
>>  _3_cut_down_F_Septile
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.22897196261682243  PRECISION:  0.05888007690459024  F1:  0.09367233798508888  GINI:  0.006041301111642028
//////////////////////////////////////////////////


In [171]:
var = vars_top[82]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COVER_GAR_AUTO_HIPO_PREN_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.241941,missing,True
1,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[-inf,0.2)",723,0.406637,681,42,0.058091,-0.055744,0.001232,0.241941,0.2,False
2,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[0.2,0.5)",373,0.209786,344,29,0.077748,0.256803,0.015488,0.241941,0.5,False
3,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[0.5,0.8)",210,0.118110,189,21,0.100000,0.532924,0.042455,0.241941,0.8,False
4,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[0.8,1.1)",128,0.071991,125,3,0.023438,-0.999553,0.047281,0.241941,1.1,False
5,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[1.1,1.6)",131,0.073678,121,10,0.076336,0.236943,0.004590,0.241941,1.6,False
6,COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"[1.6,inf)",119,0.066929,116,3,0.025210,-0.924829,0.038762,0.241941,inf,False


In [172]:
var = vars_top[83]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF,"[-inf,2.0)",1681,0.945444,1594,87,0.051755,-0.169646,0.025266,0.241556,2.0,False
1,ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF,"[2.0,inf)",97,0.054556,76,21,0.216495,1.452237,0.216290,0.241556,inf,False


In [173]:
var = vars_top[84]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_COMEX_VIGENTE_U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_COMEX_VIGENTE_U2M,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.241208,missing,True
1,MONTO_COMEX_VIGENTE_U2M,"[-inf,500000.0)",1112,0.625422,1056,56,0.050360,-0.206743,0.024417,0.241208,500000.0,False
2,MONTO_COMEX_VIGENTE_U2M,"[500000.0,6000000.0)",235,0.132171,215,20,0.085106,0.355243,0.019507,0.241208,6000000.0,False
3,MONTO_COMEX_VIGENTE_U2M,"[6000000.0,11000000.0)",110,0.061867,106,4,0.036364,-0.546996,0.014628,0.241208,11000000.0,False
4,MONTO_COMEX_VIGENTE_U2M,"[11000000.0,15500000.0)",89,0.050056,80,9,0.101124,0.545347,0.018946,0.241208,15500000.0,False
5,MONTO_COMEX_VIGENTE_U2M,"[15500000.0,inf)",145,0.081552,126,19,0.131034,0.838306,0.083011,0.241208,inf,False


In [174]:
var = vars_top[85]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,missing,220,0.123735,206,14,0.063636,0.049629,0.000311,0.240598,missing,True
1,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[-inf,-55.0)",139,0.078178,127,12,0.086331,0.379167,0.013295,0.240598,-55.0,False
2,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[-55.0,-30.0)",108,0.060742,106,2,0.018519,-1.231844,0.055377,0.240598,-30.0,False
3,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[-30.0,-15.0)",129,0.072553,118,11,0.085271,0.365658,0.011406,0.240598,-15.0,False
4,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[-15.0,-10.0)",107,0.060180,105,2,0.018692,-1.222365,0.054219,0.240598,-10.0,False
5,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[-10.0,0.0)",170,0.095613,157,13,0.076471,0.247151,0.006515,0.240598,0.0,False
6,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[0.0,65.0)",774,0.435321,737,37,0.047804,-0.253222,0.024999,0.240598,65.0,False
7,ROTACION_CTASXPAGAR_VAR_DIAS_CEF_SITUACION,"[65.0,inf)",131,0.073678,114,17,0.129771,0.835463,0.074476,0.240598,inf,False


In [175]:
data = add_new_var_x_cut_up(data, colum='NRO_MESES_SIN_LINEA_VIGENTE', cortes=[1, 3, 4], impute=0, up=False, soft=True)

>>  _1_cut_down_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3429906542056075  PRECISION:  0.05263157894736842  F1:  0.09125948029342285  GINI:  -0.033076748731885486
//////////////////////////////////////////////////
>>  _3_cut_down_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4224299065420561  PRECISION:  0.057763578274760384  F1:  0.1016301292861158  GINI:  0.0026986074415209504
//////////////////////////////////////////////////
>>  _4_cut_down_NUM_MESES_SIN_LINEA_VIGENTE
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4691588785046729  PRECISION:  0.06092233009708738  F1:  0.10784103114930181  GINI:  0.028648802220943015
//////////////////////////////////////////////////


In [176]:
var = vars_top[86]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCETN_DEBT_IN__SCOTIASF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCETN_DEBT_IN__SCOTIASF_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.236969,missing,True
1,PERCETN_DEBT_IN__SCOTIASF_ULT_RCC,"[-inf,0.1)",1214,0.682790,1130,84,0.069193,0.130993,0.012405,0.236969,0.1,False
2,PERCETN_DEBT_IN__SCOTIASF_ULT_RCC,"[0.1,0.30000000000000004)",223,0.125422,204,19,0.085202,0.356468,0.018649,0.236969,0.30000000000000004,False
3,PERCETN_DEBT_IN__SCOTIASF_ULT_RCC,"[0.30000000000000004,inf)",247,0.138920,242,5,0.020243,-1.149351,0.113782,0.236969,inf,False


In [177]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_auto_hipo_pren_u12m', cortes=[0.3, 0.4, 0.5], impute=0, up=False, soft=True)

>>  _0.3_cut_down_minimo_percent_auto_hipo_pren_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6317757009345795  PRECISION:  0.05526036131774708  F1:  0.10163121100503647  GINI:  -0.026143005657701113
//////////////////////////////////////////////////
>>  _0.4_cut_down_minimo_percent_auto_hipo_pren_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7383177570093458  PRECISION:  0.05917159763313609  F1:  0.1095624436585535  GINI:  0.023243181124112944
//////////////////////////////////////////////////
>>  _0.5_cut_down_minimo_percent_auto_hipo_pren_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.805607476635514  PRECISION:  0.06031346207668626  F1:  0.1122249707069392  GINI:  0.04106233260727765
//////////////////////////////////////////////////


In [178]:
var = vars_top[87]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U6M_PASADO_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U6M_PASADO_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.236094,missing,True
1,ADEX_MAX_U6M_PASADO_IMP,"[-inf,100000.0)",520,0.292463,495,25,0.048077,-0.247234,0.016052,0.236094,100000.0,False
2,ADEX_MAX_U6M_PASADO_IMP,"[100000.0,400000.0)",288,0.161980,279,9,0.031250,-0.695540,0.058239,0.236094,400000.0,False
3,ADEX_MAX_U6M_PASADO_IMP,"[400000.0,700000.0)",147,0.082677,128,19,0.129252,0.830856,0.082487,0.236094,700000.0,False
4,ADEX_MAX_U6M_PASADO_IMP,"[700000.0,2800000.0)",411,0.231159,392,19,0.046229,-0.288375,0.016958,0.236094,2800000.0,False
5,ADEX_MAX_U6M_PASADO_IMP,"[2800000.0,inf)",124,0.069741,109,15,0.120968,0.755150,0.055594,0.236094,inf,False


In [179]:
var = vars_top[88]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COVER_GAR_AUTO_HIPO_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COVER_GAR_AUTO_HIPO_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.235165,missing,True
1,COVER_GAR_AUTO_HIPO_ULT_RCC,"[-inf,0.2)",817,0.459505,770,47,0.057528,-0.066094,0.001949,0.235165,0.2,False
2,COVER_GAR_AUTO_HIPO_ULT_RCC,"[0.2,0.5)",378,0.212598,349,29,0.076720,0.242373,0.013892,0.235165,0.5,False
3,COVER_GAR_AUTO_HIPO_ULT_RCC,"[0.5,0.8)",162,0.091114,142,20,0.123457,0.770054,0.075946,0.235165,0.8,False
4,COVER_GAR_AUTO_HIPO_ULT_RCC,"[0.8,1.5)",202,0.113611,196,6,0.029703,-0.756206,0.047088,0.235165,1.5,False
5,COVER_GAR_AUTO_HIPO_ULT_RCC,"[1.5,inf)",125,0.070304,119,6,0.048000,-0.257215,0.004157,0.235165,inf,False


In [180]:
var = vars_top[89]
print(">>>>> ", var)
display(cortes[var])

>>>>>  cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.234787,missing,True
1,cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,sobre cubierto en 20%,99,0.055681,97,2,0.020202,-1.151415,0.045732,0.234787,sobre cubierto en 20%,False
2,cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"sobre cubierto de 20% a 40%%,%sobre cubierto a mas del 60%",160,0.089989,156,4,0.025000,-0.933413,0.052908,0.234787,"sobre cubierto de 20% a 40%%,%sobre cubierto a mas del 60%",False
3,cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,menos del 10%,615,0.345894,578,37,0.060163,-0.018507,0.000117,0.234787,menos del 10%,False
4,cat_COVER_GAR_AUTO_HIPO_PREN_ULT_RCC,"entre10 y 20%%%,%sobre cubierto de 40% a 60%",810,0.455568,745,65,0.080247,0.291152,0.043897,0.234787,"entre10 y 20%%%,%sobre cubierto de 40% a 60%",False


In [181]:
var = vars_top[90]
print(">>>>> ", var)
display(cortes[var])

>>>>>  cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.233223,missing,True
1,cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,sobre cubierto en 20%,99,0.055681,97,2,0.020202,-1.151415,0.045732,0.233223,sobre cubierto en 20%,False
2,cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,"sobre cubierto de 20% a 40%%,%sobre cubierto a mas del 60%",160,0.089989,156,4,0.025000,-0.933413,0.052908,0.233223,"sobre cubierto de 20% a 40%%,%sobre cubierto a mas del 60%",False
3,cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,menos del 10%,611,0.343645,574,37,0.060556,-0.011563,0.000046,0.233223,menos del 10%,False
4,cat_COVER_GAR_AUTO_HIPO_PREN_NOPREFER_ULT_RCC,"entre10 y 20%%%,%sobre cubierto de 40% a 60%",814,0.457818,749,65,0.079853,0.285797,0.042405,0.233223,"entre10 y 20%%%,%sobre cubierto de 40% a 60%",False


In [182]:
data = add_new_var_x_cut_up(data, colum='minimo_percent_sow_ibk_u12m', cortes=[0.6, 0.2], impute=0, up=False, soft=True)

>>  _0.6_cut_down_minimo_percent_sow_ibk_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.8392523364485981  PRECISION:  0.06451149425287356  F1:  0.11981320880587057  GINI:  0.09793388034020678
//////////////////////////////////////////////////
>>  _0.2_cut_down_minimo_percent_sow_ibk_u12m
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6728971962616822  PRECISION:  0.061633281972265024  F1:  0.11292346298619825  GINI:  0.04885074288584268
//////////////////////////////////////////////////


In [183]:
var = vars_top[91]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_CASH_IN_U12M_2


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_CASH_IN_U12M_2,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.231945,missing,True
1,MTO_CASH_IN_U12M_2,"[-inf,200000.0)",648,0.364454,626,22,0.033951,-0.609860,0.104376,0.231945,200000.0,False
2,MTO_CASH_IN_U12M_2,"[200000.0,500000.0)",242,0.136108,224,18,0.074380,0.217173,0.007066,0.231945,500000.0,False
3,MTO_CASH_IN_U12M_2,"[500000.0,3200000.0)",549,0.308774,496,53,0.096539,0.502164,0.097287,0.231945,3200000.0,False
4,MTO_CASH_IN_U12M_2,"[3200000.0,inf)",248,0.139483,236,12,0.048387,-0.240477,0.007264,0.231945,inf,False


In [184]:
data = add_new_var_x_cut_up(data, colum='NRO_FEVE_SEGUIR_GRUPO_EECON', cortes=[1], impute=0, up=False, soft=True)

>>  _1_cut_down_NUM_FEVE_SEGUIR_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7775700934579439  PRECISION:  0.060180831826401444  F1:  0.11171534071836185  GINI:  0.037902553892875
//////////////////////////////////////////////////


In [185]:
var = vars_top[92]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_CASH_IN_12UM


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_CASH_IN_12UM,"[-inf,200000.0)",739,0.415636,714,25,0.033829,-0.613559,0.120297,0.231913,200000.0,False
1,MTO_CASH_IN_12UM,"[200000.0,500000.0)",242,0.136108,224,18,0.074380,0.217173,0.007066,0.231913,500000.0,False
2,MTO_CASH_IN_12UM,"[500000.0,3200000.0)",549,0.308774,496,53,0.096539,0.502164,0.097287,0.231913,3200000.0,False
3,MTO_CASH_IN_12UM,"[3200000.0,inf)",248,0.139483,236,12,0.048387,-0.240477,0.007264,0.231913,inf,False


In [186]:
var = vars_top[93]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ROTACION_CTASXPAGAR_VAR_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,missing,179,0.100675,169,10,0.055866,-0.088866,0.000765,0.231557,missing,True
1,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[-inf,-0.7000000000000001)",92,0.051744,88,4,0.043478,-0.352595,0.005521,0.231557,-0.7000000000000001,False
2,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[-0.7000000000000001,-0.40000000000000013)",230,0.129359,208,22,0.095652,0.491952,0.038939,0.231557,-0.40000000000000013,False
3,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[-0.40000000000000013,-0.1000000000000002)",245,0.137795,236,9,0.036735,-0.528160,0.030625,0.231557,-0.1000000000000002,False
4,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[-0.1000000000000002,0.49999999999999956)",600,0.337458,564,36,0.060000,-0.013088,0.000057,0.231557,0.49999999999999956,False
5,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[0.49999999999999956,1.0999999999999996)",223,0.125422,218,5,0.022422,-1.036609,0.087327,0.231557,1.0999999999999996,False
6,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[1.0999999999999996,1.9999999999999991)",106,0.059618,92,14,0.132075,0.855716,0.063785,0.231557,1.9999999999999991,False
7,ROTACION_CTASXPAGAR_VAR_CEF_SITUACION,"[1.9999999999999991,inf)",103,0.057930,95,8,0.077670,0.264012,0.004538,0.231557,inf,False


In [187]:
var = vars_top[94]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTOSALDO_IBK_V_U1M_OVER_U6M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTOSALDO_IBK_V_U1M_OVER_U6M,missing,49,0.027559,49,0,0.000000,-1.267032,0.026705,0.231523,missing,True
1,MTOSALDO_IBK_V_U1M_OVER_U6M,"[-inf,0.4)",532,0.299213,513,19,0.035714,-0.565688,0.075075,0.231523,0.4,False
2,MTOSALDO_IBK_V_U1M_OVER_U6M,"[0.4,0.9)",292,0.164229,267,25,0.085616,0.361776,0.025212,0.231523,0.9,False
3,MTOSALDO_IBK_V_U1M_OVER_U6M,"[0.9,1.0)",151,0.084927,131,20,0.132450,0.850684,0.089502,0.231523,1.0,False
4,MTOSALDO_IBK_V_U1M_OVER_U6M,"[1.0,1.3)",488,0.274466,456,32,0.065574,0.073392,0.001526,0.231523,1.3,False
5,MTOSALDO_IBK_V_U1M_OVER_U6M,"[1.3,inf)",266,0.149606,254,12,0.045113,-0.322279,0.013504,0.231523,inf,False


In [188]:
var = vars_top[95]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_DESCUENTOS_VIGENTE_U2M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_DESCUENTOS_VIGENTE_U2M,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.230728,missing,True
1,MONTO_DESCUENTOS_VIGENTE_U2M,"[-inf,400000.0)",1198,0.673791,1128,70,0.058431,-0.049557,0.001618,0.230728,400000.0,False
2,MONTO_DESCUENTOS_VIGENTE_U2M,"[400000.0,1650000.0)",161,0.090551,154,7,0.043478,-0.360894,0.010082,0.230728,1650000.0,False
3,MONTO_DESCUENTOS_VIGENTE_U2M,"[1650000.0,3050000.0)",90,0.050619,88,2,0.022222,-1.054041,0.036184,0.230728,3050000.0,False
4,MONTO_DESCUENTOS_VIGENTE_U2M,"[3050000.0,inf)",242,0.136108,213,29,0.119835,0.736153,0.102144,0.230728,inf,False


In [189]:
var = vars_top[96]
print(">>>>> ", var)
display(cortes[var])

>>>>>  minimo_percent_sow_others_u12m


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,minimo_percent_sow_others_u12m,missing,18,0.010124,18,0,0.000000,-0.265583,0.000668,0.228105,missing,True
1,minimo_percent_sow_others_u12m,"[-inf,0.4)",552,0.310461,515,37,0.067029,0.096900,0.003041,0.228105,0.4,False
2,minimo_percent_sow_others_u12m,"[0.4,0.7000000000000001)",460,0.258718,422,38,0.082609,0.322730,0.031063,0.228105,0.7000000000000001,False
3,minimo_percent_sow_others_u12m,"[0.7000000000000001,0.8)",219,0.123172,213,6,0.027397,-0.839384,0.060812,0.228105,0.8,False
4,minimo_percent_sow_others_u12m,"[0.8,0.9)",246,0.138358,227,19,0.077236,0.249638,0.009622,0.228105,0.9,False
5,minimo_percent_sow_others_u12m,"[0.9,1.0)",175,0.098425,168,7,0.040000,-0.447905,0.016268,0.228105,1.0,False
6,minimo_percent_sow_others_u12m,"[1.0,inf)",108,0.060742,107,1,0.009259,-1.942680,0.106632,0.228105,inf,False


In [190]:
var = vars_top[97]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NUMERO_TRABAJADORES_REGISTRADOS_U10M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NUMERO_TRABAJADORES_REGISTRADOS_U10M,missing,87,0.048931,83,4,0.045977,-0.294099,0.003724,0.227348,missing,True
1,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[-inf,20.0)",501,0.281777,467,34,0.067864,0.118479,0.004167,0.227348,20.0,False
2,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[20.0,60.0)",125,0.070304,123,2,0.016000,-1.380589,0.076118,0.227348,60.0,False
3,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[60.0,120.0)",299,0.168166,282,17,0.056856,-0.070246,0.000805,0.227348,120.0,False
4,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[120.0,160.0)",125,0.070304,122,3,0.024000,-0.966961,0.043780,0.227348,160.0,False
5,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[160.0,220.0)",172,0.096738,160,12,0.069767,0.148181,0.002268,0.227348,220.0,False
6,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[220.0,320.0)",148,0.083240,130,18,0.121622,0.761285,0.067619,0.227348,320.0,False
7,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[320.0,540.0)",147,0.082677,142,5,0.034014,-0.607941,0.023548,0.227348,540.0,False
8,NUMERO_TRABAJADORES_REGISTRADOS_U10M,"[540.0,inf)",174,0.097863,161,13,0.074713,0.221993,0.005320,0.227348,inf,False


In [191]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M', cortes=[2, 3, 4, 5], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3168224299065421  PRECISION:  0.08300685602350637  F1:  0.13154831199068684  GINI:  0.10362648319129675
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.07991940899932841  F1:  0.09300508010941774  GINI:  0.03322337863821456
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.07196261682242991  PRECISION:  0.08546059933407325  F1:  0.07813292744799594  GINI:  0.025053815729409346
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.020560747663551402  PRECISION:  0.06162464985994398  F1:  0.03083391730903995  G

In [192]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M', cortes=[2], impute=0, up=False, soft=True)

>>  _2_cut_down_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.6831775700934579  PRECISION:  0.05023364485981308  F1:  0.09358596850595313  GINI:  -0.10362648319129675
//////////////////////////////////////////////////


In [193]:
var = vars_top[98]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,missing,169,0.095051,159,10,0.059172,-0.027871,0.000073,0.226061,missing,True
1,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,"[-inf,-0.6999999999999995)",95,0.053431,93,2,0.021053,-1.101005,0.040924,0.226061,-0.6999999999999995,False
2,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,"[-0.6999999999999995,-0.09999999999999898)",355,0.199663,338,17,0.047887,-0.251385,0.011309,0.226061,-0.09999999999999898,False
3,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,"[-0.09999999999999898,1.1102230246251565e-15)",150,0.084364,133,17,0.113333,0.681312,0.052983,0.226061,1.1102230246251565e-15,False
4,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,"[1.1102230246251565e-15,0.7000000000000017)",704,0.395951,666,38,0.053977,-0.125256,0.005881,0.226061,0.7000000000000017,False
5,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,"[0.7000000000000017,2.200000000000003)",212,0.119235,203,9,0.042453,-0.377534,0.014431,0.226061,2.200000000000003,False
6,ROTACION_CTASXCOBRAR_VAR_CEF_SITUACION,"[2.200000000000003,inf)",93,0.052306,78,15,0.161290,1.089789,0.100459,0.226061,inf,False


In [194]:
#data = add_new_var_x_cut_up(data, colum='MAX_DIAS_ATRASO_SFSF_ULT_RCC', cortes=[4], impute=0, up=False, soft=True)

In [195]:
var = vars_top[99]
print(">>>>> ", var)
display(cortes[var])

>>>>>  NRO_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF,"[-inf,1.0)",1637,0.920697,1554,83,0.050703,-0.191299,0.030994,0.226024,1.0,False
1,NRO_MESES_CON_ATRASO_4diasomas_U6M_COLDIRECT_SF,"[1.0,inf)",141,0.079303,116,25,0.177305,1.203733,0.195029,0.226024,inf,False


In [196]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M', cortes=[2, 3, 4, 5], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3168224299065421  PRECISION:  0.08300685602350637  F1:  0.13154831199068684  GINI:  0.10362648319129675
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11121495327102804  PRECISION:  0.07991940899932841  F1:  0.09300508010941774  GINI:  0.03322337863821456
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.07196261682242991  PRECISION:  0.08546059933407325  F1:  0.07813292744799594  GINI:  0.025053815729409346
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_BANCOS_NOIBK_DEBT_MAYOR_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.020560747663551402  PRECISION:  0.06162464985994398  F1:  0.03083391730903995  G

In [197]:
var = vars_top[100]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.224242,missing,True
1,MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x,"[-inf,2.0)",1536,0.863892,1450,86,0.055990,-0.094823,0.007448,0.224242,2.0,False
2,MAX_DIAS_ATRASO_BCPSF_ULT_RCC_x,"[2.0,inf)",148,0.083240,126,22,0.148649,0.984909,0.124661,0.224242,inf,False


In [198]:
var = vars_top[101]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.223855,missing,True
1,DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT,"[-inf,-8.0)",267,0.150169,232,35,0.131086,0.847058,0.156834,0.223855,-8.0,False
2,DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT,"[-8.0,-4.0)",105,0.059055,97,8,0.076190,0.243178,0.003888,0.223855,-4.0,False
3,DIFF_PROM_CANT_TRX_U6M_U12M_CASHOUT,"[-4.0,inf)",1315,0.739595,1253,62,0.047148,-0.267714,0.047178,0.223855,inf,False


In [199]:
var = vars_top[102]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PROM_CANT_TRX_U6M_CASHOUT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PROM_CANT_TRX_U6M_CASHOUT,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.223683,missing,True
1,PROM_CANT_TRX_U6M_CASHOUT,"[-inf,10.0)",286,0.160855,269,17,0.059441,-0.023050,0.000085,0.223683,10.0,False
2,PROM_CANT_TRX_U6M_CASHOUT,"[10.0,20.0)",218,0.122610,214,4,0.018349,-1.241234,0.113085,0.223683,20.0,False
3,PROM_CANT_TRX_U6M_CASHOUT,"[20.0,25.0)",104,0.058493,100,4,0.038462,-0.480428,0.010975,0.223683,25.0,False
4,PROM_CANT_TRX_U6M_CASHOUT,"[25.0,120.0)",803,0.451631,744,59,0.073474,0.203944,0.020555,0.223683,120.0,False
5,PROM_CANT_TRX_U6M_CASHOUT,"[120.0,220.0)",159,0.089426,142,17,0.106918,0.615834,0.044573,0.223683,220.0,False
6,PROM_CANT_TRX_U6M_CASHOUT,"[220.0,inf)",117,0.065804,113,4,0.034188,-0.602646,0.018458,0.223683,inf,False


In [200]:
var = vars_top[103]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.222319,missing,True
1,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[-inf,0.1)",737,0.414511,698,39,0.052917,-0.154509,0.009245,0.222319,0.1,False
2,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[0.1,0.30000000000000004)",299,0.168166,275,24,0.080268,0.291432,0.016237,0.222319,0.30000000000000004,False
3,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[0.30000000000000004,0.6000000000000001)",268,0.150731,251,17,0.063433,0.037909,0.000220,0.222319,0.6000000000000001,False
4,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[0.6000000000000001,0.8)",89,0.050056,87,2,0.022472,-1.042612,0.035168,0.222319,0.8,False
5,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[0.8,1.0)",133,0.074803,126,7,0.052632,-0.160223,0.001790,0.222319,1.0,False
6,PERCENT_DIRECTA_VIGVENC_LINMER_IBK_V,"[1.0,inf)",158,0.088864,139,19,0.120253,0.740114,0.067527,0.222319,inf,False


In [201]:
var = vars_top[104]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCETN_DEBT_IN__IBKSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCETN_DEBT_IN__IBKSF_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.222319,missing,True
1,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[-inf,0.1)",737,0.414511,698,39,0.052917,-0.154509,0.009245,0.222319,0.1,False
2,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[0.1,0.30000000000000004)",299,0.168166,275,24,0.080268,0.291432,0.016237,0.222319,0.30000000000000004,False
3,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[0.30000000000000004,0.6000000000000001)",268,0.150731,251,17,0.063433,0.037909,0.000220,0.222319,0.6000000000000001,False
4,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[0.6000000000000001,0.8)",89,0.050056,87,2,0.022472,-1.042612,0.035168,0.222319,0.8,False
5,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[0.8,1.0)",133,0.074803,126,7,0.052632,-0.160223,0.001790,0.222319,1.0,False
6,PERCETN_DEBT_IN__IBKSF_ULT_RCC,"[1.0,inf)",158,0.088864,139,19,0.120253,0.740114,0.067527,0.222319,inf,False


In [202]:
var = vars_top[105]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_MAX_U3M_PASADO_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_MAX_U3M_PASADO_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.222287,missing,True
1,ADEX_MAX_U3M_PASADO_IMP,"[-inf,100000.0)",605,0.340270,578,27,0.044628,-0.325289,0.031263,0.222287,100000.0,False
2,ADEX_MAX_U3M_PASADO_IMP,"[100000.0,200000.0)",132,0.074241,128,4,0.030303,-0.727288,0.028808,0.222287,200000.0,False
3,ADEX_MAX_U3M_PASADO_IMP,"[200000.0,700000.0)",274,0.154106,249,25,0.091241,0.439871,0.036236,0.222287,700000.0,False
4,ADEX_MAX_U3M_PASADO_IMP,"[700000.0,1500000.0)",203,0.114173,197,6,0.029557,-0.752997,0.046993,0.222287,1500000.0,False
5,ADEX_MAX_U3M_PASADO_IMP,"[1500000.0,2700000.0)",176,0.098988,165,11,0.062500,0.030397,0.000093,0.222287,2700000.0,False
6,ADEX_MAX_U3M_PASADO_IMP,"[2700000.0,inf)",100,0.056243,86,14,0.140000,0.923158,0.072129,0.222287,inf,False


In [203]:
var = vars_top[106]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_IN__BCPSF_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_IN__BCPSF_ULT_RCC,missing,99,0.055681,99,0,0.000000,-1.970331,0.100520,0.219271,missing,True
1,DEBT_IN__BCPSF_ULT_RCC,"[-inf,7000000.0)",1155,0.649606,1087,68,0.058874,-0.041520,0.001099,0.219271,7000000.0,False
2,DEBT_IN__BCPSF_ULT_RCC,"[7000000.0,10000000.0)",120,0.067492,116,4,0.033333,-0.637147,0.020854,0.219271,10000000.0,False
3,DEBT_IN__BCPSF_ULT_RCC,"[10000000.0,16000000.0)",177,0.099550,168,9,0.050847,-0.196591,0.003530,0.219271,16000000.0,False
4,DEBT_IN__BCPSF_ULT_RCC,"[16000000.0,inf)",227,0.127672,200,27,0.118943,0.727668,0.093268,0.219271,inf,False


In [204]:
var = vars_top[107]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_FULL_RCC_SANT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_FULL_RCC_SANT,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.21776,missing,True
1,MTO_DEUDA_FULL_RCC_SANT,"[-inf,1.0)",1321,0.742970,1262,59,0.044663,-0.332767,0.071181,0.21776,1.0,False
2,MTO_DEUDA_FULL_RCC_SANT,"[1.0,inf)",456,0.256468,407,49,0.107456,0.613156,0.126458,0.21776,inf,False


In [205]:
var = vars_top[108]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_BCP_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_BCP_ULT_RCC,missing,18,0.010124,18,0,0.000000,-0.265583,0.000668,0.214824,missing,True
1,SOW_BCP_ULT_RCC,"[-inf,0.1)",760,0.427447,717,43,0.056579,-0.083727,0.002887,0.214824,0.1,False
2,SOW_BCP_ULT_RCC,"[0.1,0.2)",228,0.128234,206,22,0.096491,0.493315,0.038808,0.214824,0.2,False
3,SOW_BCP_ULT_RCC,"[0.2,0.30000000000000004)",151,0.084927,146,5,0.033113,-0.644020,0.026734,0.214824,0.30000000000000004,False
4,SOW_BCP_ULT_RCC,"[0.30000000000000004,0.5)",289,0.162542,274,15,0.051903,-0.174929,0.004606,0.214824,0.5,False
5,SOW_BCP_ULT_RCC,"[0.5,0.8)",227,0.127672,205,22,0.096916,0.498181,0.039489,0.214824,0.8,False
6,SOW_BCP_ULT_RCC,"[0.8,inf)",105,0.059055,104,1,0.009524,-1.914242,0.101632,0.214824,inf,False


In [206]:
var = vars_top[109]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TRX_CASHIN_TRANSFERENCIAS_U3M_MASIVO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TRX_CASHIN_TRANSFERENCIAS_U3M_MASIVO,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.212896,missing,True
1,TRX_CASHIN_TRANSFERENCIAS_U3M_MASIVO,"[-inf,340000.0)",1357,0.763217,1280,77,0.056743,-0.072362,0.003872,0.212896,340000.0,False
2,TRX_CASHIN_TRANSFERENCIAS_U3M_MASIVO,"[340000.0,520000.0)",94,0.052868,79,15,0.159574,1.077050,0.098640,0.212896,520000.0,False
3,TRX_CASHIN_TRANSFERENCIAS_U3M_MASIVO,"[520000.0,1040000.0)",113,0.063555,102,11,0.097345,0.511370,0.020851,0.212896,1040000.0,False
4,TRX_CASHIN_TRANSFERENCIAS_U3M_MASIVO,"[1040000.0,inf)",123,0.069179,121,2,0.016260,-1.364196,0.073580,0.212896,inf,False


In [207]:
var = vars_top[110]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TRX_CASHOUT_TOTAL_1UM_MASIVO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TRX_CASHOUT_TOTAL_1UM_MASIVO,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.212773,missing,True
1,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[-inf,200000.0)",722,0.406074,681,41,0.056787,-0.071543,0.002014,0.212773,200000.0,False
2,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[200000.0,600000.0)",242,0.136108,227,15,0.061983,0.021548,0.000064,0.212773,600000.0,False
3,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[600000.0,1200000.0)",180,0.101237,175,5,0.027778,-0.816900,0.047784,0.212773,1200000.0,False
4,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[1200000.0,2000000.0)",140,0.078740,121,19,0.135714,0.887096,0.091789,0.212773,2000000.0,False
5,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[2000000.0,3600000.0)",159,0.089426,150,9,0.056604,-0.074963,0.000486,0.212773,3600000.0,False
6,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[3600000.0,6400000.0)",99,0.055681,88,11,0.111111,0.659006,0.032395,0.212773,6400000.0,False
7,TRX_CASHOUT_TOTAL_1UM_MASIVO,"[6400000.0,inf)",145,0.081552,140,5,0.034483,-0.593757,0.022287,0.212773,inf,False


In [208]:
data = add_new_var_x_cut_up(data, colum='RECENCY_historia', cortes=[32], impute=0, up=True, soft=False)

>>  _32_cut_up_RECENCY_historia
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.0794392523364486  PRECISION:  0.06041222459132907  F1:  0.06863140896245458  GINI:  0.004180229223617049
//////////////////////////////////////////////////


In [209]:
var = vars_top[111]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_CASHOUT_TOTAL_MES_ACT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_CASHOUT_TOTAL_MES_ACT,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.212773,missing,True
1,MTO_CASHOUT_TOTAL_MES_ACT,"[-inf,200000.0)",722,0.406074,681,41,0.056787,-0.071543,0.002014,0.212773,200000.0,False
2,MTO_CASHOUT_TOTAL_MES_ACT,"[200000.0,600000.0)",242,0.136108,227,15,0.061983,0.021548,0.000064,0.212773,600000.0,False
3,MTO_CASHOUT_TOTAL_MES_ACT,"[600000.0,1200000.0)",180,0.101237,175,5,0.027778,-0.816900,0.047784,0.212773,1200000.0,False
4,MTO_CASHOUT_TOTAL_MES_ACT,"[1200000.0,2000000.0)",140,0.078740,121,19,0.135714,0.887096,0.091789,0.212773,2000000.0,False
5,MTO_CASHOUT_TOTAL_MES_ACT,"[2000000.0,3600000.0)",159,0.089426,150,9,0.056604,-0.074963,0.000486,0.212773,3600000.0,False
6,MTO_CASHOUT_TOTAL_MES_ACT,"[3600000.0,6400000.0)",99,0.055681,88,11,0.111111,0.659006,0.032395,0.212773,6400000.0,False
7,MTO_CASHOUT_TOTAL_MES_ACT,"[6400000.0,inf)",145,0.081552,140,5,0.034483,-0.593757,0.022287,0.212773,inf,False


In [210]:
var = vars_top[112]
print(">>>>> ", var)
display(cortes[var])

>>>>>  _aux_MONTO_CASHOUT_TOTAL_U1M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,_aux_MONTO_CASHOUT_TOTAL_U1M,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.212773,missing,True
1,_aux_MONTO_CASHOUT_TOTAL_U1M,"[-inf,200000.0)",722,0.406074,681,41,0.056787,-0.071543,0.002014,0.212773,200000.0,False
2,_aux_MONTO_CASHOUT_TOTAL_U1M,"[200000.0,600000.0)",242,0.136108,227,15,0.061983,0.021548,0.000064,0.212773,600000.0,False
3,_aux_MONTO_CASHOUT_TOTAL_U1M,"[600000.0,1200000.0)",180,0.101237,175,5,0.027778,-0.816900,0.047784,0.212773,1200000.0,False
4,_aux_MONTO_CASHOUT_TOTAL_U1M,"[1200000.0,2000000.0)",140,0.078740,121,19,0.135714,0.887096,0.091789,0.212773,2000000.0,False
5,_aux_MONTO_CASHOUT_TOTAL_U1M,"[2000000.0,3600000.0)",159,0.089426,150,9,0.056604,-0.074963,0.000486,0.212773,3600000.0,False
6,_aux_MONTO_CASHOUT_TOTAL_U1M,"[3600000.0,6400000.0)",99,0.055681,88,11,0.111111,0.659006,0.032395,0.212773,6400000.0,False
7,_aux_MONTO_CASHOUT_TOTAL_U1M,"[6400000.0,inf)",145,0.081552,140,5,0.034483,-0.593757,0.022287,0.212773,inf,False


In [211]:
var = vars_top[113]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MTO_DEUDA_FULL_RCC_BIF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MTO_DEUDA_FULL_RCC_BIF,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.212628,missing,True
1,MTO_DEUDA_FULL_RCC_BIF,"[-inf,800000.0)",1090,0.613048,1035,55,0.050459,-0.204675,0.023478,0.212628,800000.0,False
2,MTO_DEUDA_FULL_RCC_BIF,"[800000.0,2000000.0)",95,0.053431,93,2,0.021053,-1.109303,0.041403,0.212628,2000000.0,False
3,MTO_DEUDA_FULL_RCC_BIF,"[2000000.0,3200000.0)",103,0.057930,96,7,0.067961,0.111711,0.000759,0.212628,3200000.0,False
4,MTO_DEUDA_FULL_RCC_BIF,"[3200000.0,4600000.0)",105,0.059055,95,10,0.095238,0.478857,0.016732,0.212628,4600000.0,False
5,MTO_DEUDA_FULL_RCC_BIF,"[4600000.0,8000000.0)",102,0.057368,99,3,0.029412,-0.766359,0.024319,0.212628,8000000.0,False
6,MTO_DEUDA_FULL_RCC_BIF,"[8000000.0,inf)",282,0.158605,251,31,0.109929,0.638683,0.085817,0.212628,inf,False


In [212]:
var = vars_top[114]
print(">>>>> ", var)
display(cortes[var])

>>>>>  COD_GRUPO_ECONOMICO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,COD_GRUPO_ECONOMICO,missing,37,0.020810,34,3,0.081081,0.310699,0.002305,0.212548,missing,True
1,COD_GRUPO_ECONOMICO,"[-inf,1600.0)",455,0.255906,422,33,0.072527,0.189950,0.010041,0.212548,1600.0,False
2,COD_GRUPO_ECONOMICO,"[1600.0,1900.0)",90,0.050619,88,2,0.022222,-1.045742,0.035739,0.212548,1900.0,False
3,COD_GRUPO_ECONOMICO,"[1900.0,2400.0)",117,0.065804,104,13,0.111111,0.659006,0.038285,0.212548,2400.0,False
4,COD_GRUPO_ECONOMICO,"[2400.0,3500.0)",531,0.298650,513,18,0.033898,-0.611456,0.085921,0.212548,3500.0,False
5,COD_GRUPO_ECONOMICO,"[3500.0,4000.0)",140,0.078740,131,9,0.064286,0.060475,0.000296,0.212548,4000.0,False
6,COD_GRUPO_ECONOMICO,"[4000.0,4900.0)",238,0.133858,216,22,0.092437,0.454212,0.033776,0.212548,4900.0,False
7,COD_GRUPO_ECONOMICO,"[4900.0,inf)",170,0.095613,162,8,0.047059,-0.269707,0.006185,0.212548,inf,False


In [213]:
data = add_new_var_x_cut_up(data, colum='ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF', cortes=[1, 5], impute=0, up=True, soft=False)

>>  _1_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.11401869158878504  PRECISION:  0.18100890207715134  F1:  0.13990825688073394  GINI:  0.08259434910899466
//////////////////////////////////////////////////
>>  _5_cut_up_ATRASO_PROM_U6M_COLDIRECT_VIG_VENC_SF
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.05327102803738318  PRECISION:  0.27403846153846156  F1:  0.08920187793427231  GINI:  0.04467487638077383
//////////////////////////////////////////////////


In [214]:
var = vars_top[115]
print(">>>>> ", var)
display(cortes[var])

>>>>>  RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1,missing,220,0.123735,206,14,0.063636,0.049629,0.000311,0.210313,missing,True
1,RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1,"[-inf,0.4)",714,0.401575,684,30,0.042017,-0.388313,0.051181,0.210313,0.4,False
2,RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1,"[0.4,1.4)",536,0.301462,494,42,0.078358,0.273582,0.025465,0.210313,1.4,False
3,RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1,"[1.4,2.1)",100,0.056243,98,2,0.020000,-1.153373,0.046324,0.210313,2.1,False
4,RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1,"[2.1,3.2)",105,0.059055,100,5,0.047619,-0.257285,0.003495,0.210313,3.2,False
5,RAC_IBK6_ACTIVIDAD_CEF_SITUACION.1,"[3.2,inf)",103,0.057930,88,15,0.145631,0.969161,0.083536,0.210313,inf,False


In [215]:
var = vars_top[116]
print(">>>>> ", var)
display(cortes[var])

>>>>>  RAC_IBK6_ACTIVIDAD_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,RAC_IBK6_ACTIVIDAD_CEF_SITUACION,missing,220,0.123735,206,14,0.063636,0.049629,0.000311,0.210313,missing,True
1,RAC_IBK6_ACTIVIDAD_CEF_SITUACION,"[-inf,0.4)",714,0.401575,684,30,0.042017,-0.388313,0.051181,0.210313,0.4,False
2,RAC_IBK6_ACTIVIDAD_CEF_SITUACION,"[0.4,1.4)",536,0.301462,494,42,0.078358,0.273582,0.025465,0.210313,1.4,False
3,RAC_IBK6_ACTIVIDAD_CEF_SITUACION,"[1.4,2.1)",100,0.056243,98,2,0.020000,-1.153373,0.046324,0.210313,2.1,False
4,RAC_IBK6_ACTIVIDAD_CEF_SITUACION,"[2.1,3.2)",105,0.059055,100,5,0.047619,-0.257285,0.003495,0.210313,3.2,False
5,RAC_IBK6_ACTIVIDAD_CEF_SITUACION,"[3.2,inf)",103,0.057930,88,15,0.145631,0.969161,0.083536,0.210313,inf,False


In [216]:
var = vars_top[117]
print(">>>>> ", var)
display(cortes[var])

>>>>>  SOW_PROM_U1M_OVER_U6M_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,SOW_PROM_U1M_OVER_U6M_ULT_RCC,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.207523,missing,True
1,SOW_PROM_U1M_OVER_U6M_ULT_RCC,"[-inf,0.1)",110,0.061867,109,1,0.009091,-1.961199,0.109997,0.207523,0.1,False
2,SOW_PROM_U1M_OVER_U6M_ULT_RCC,"[0.1,0.5)",149,0.083802,144,5,0.033557,-0.630227,0.025407,0.207523,0.5,False
3,SOW_PROM_U1M_OVER_U6M_ULT_RCC,"[0.5,1.0)",518,0.291339,475,43,0.083012,0.328034,0.036224,0.207523,1.0,False
4,SOW_PROM_U1M_OVER_U6M_ULT_RCC,"[1.0,1.2000000000000002)",635,0.357143,602,33,0.051969,-0.173601,0.009973,0.207523,1.2000000000000002,False
5,SOW_PROM_U1M_OVER_U6M_ULT_RCC,"[1.2000000000000002,inf)",365,0.205287,339,26,0.071233,0.162245,0.005801,0.207523,inf,False


In [217]:
var = vars_top[118]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_HIPO_TOT


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_HIPO_TOT,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.206015,missing,True
1,GAR_HIPO_TOT,"[-inf,6000000.0)",955,0.537120,907,48,0.050262,-0.200494,0.019783,0.206015,6000000.0,False
2,GAR_HIPO_TOT,"[6000000.0,8000000.0)",93,0.052306,91,2,0.021505,-1.079265,0.038824,0.206015,8000000.0,False
3,GAR_HIPO_TOT,"[8000000.0,18000000.0)",317,0.178290,288,29,0.091483,0.442783,0.042535,0.206015,18000000.0,False
4,GAR_HIPO_TOT,"[18000000.0,24000000.0)",91,0.051181,86,5,0.054945,-0.106462,0.000554,0.206015,24000000.0,False
5,GAR_HIPO_TOT,"[24000000.0,38000000.0)",92,0.051744,78,14,0.152174,1.020796,0.084648,0.206015,38000000.0,False
6,GAR_HIPO_TOT,"[38000000.0,inf)",113,0.063555,107,6,0.053097,-0.142622,0.001215,0.206015,inf,False


In [218]:
var = vars_top[119]
print(">>>>> ", var)
display(cortes[var])

>>>>>  _aux_TRX_MONTO_TOTAL_U11M


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,_aux_TRX_MONTO_TOTAL_U11M,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.205657,missing,True
1,_aux_TRX_MONTO_TOTAL_U11M,"[-inf,200000.0)",711,0.399888,684,27,0.037975,-0.493673,0.078781,0.205657,200000.0,False
2,_aux_TRX_MONTO_TOTAL_U11M,"[200000.0,700000.0)",331,0.186164,306,25,0.075529,0.233738,0.011277,0.205657,700000.0,False
3,_aux_TRX_MONTO_TOTAL_U11M,"[700000.0,3200000.0)",429,0.241282,386,43,0.100233,0.543810,0.090822,0.205657,3200000.0,False
4,_aux_TRX_MONTO_TOTAL_U11M,"[3200000.0,inf)",216,0.121485,206,10,0.046296,-0.286843,0.008824,0.205657,inf,False


In [219]:
var = vars_top[120]
print(">>>>> ", var)
display(cortes[var])

>>>>>  ADEX_PROM_U6M_ACTUAL_IMP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,ADEX_PROM_U6M_ACTUAL_IMP,missing,288,0.161980,267,21,0.072917,0.195721,0.006765,0.205229,missing,True
1,ADEX_PROM_U6M_ACTUAL_IMP,"[-inf,50000.0)",623,0.350394,593,30,0.048154,-0.245549,0.018984,0.205229,50000.0,False
2,ADEX_PROM_U6M_ACTUAL_IMP,"[50000.0,100000.0)",100,0.056243,98,2,0.020000,-1.153373,0.046324,0.205229,100000.0,False
3,ADEX_PROM_U6M_ACTUAL_IMP,"[100000.0,300000.0)",190,0.106862,169,21,0.110526,0.653071,0.060897,0.205229,300000.0,False
4,ADEX_PROM_U6M_ACTUAL_IMP,"[300000.0,450000.0)",133,0.074803,130,3,0.022556,-1.030474,0.051592,0.205229,450000.0,False
5,ADEX_PROM_U6M_ACTUAL_IMP,"[450000.0,1450000.0)",277,0.155793,261,16,0.057762,-0.053484,0.000435,0.205229,1450000.0,False
6,ADEX_PROM_U6M_ACTUAL_IMP,"[1450000.0,inf)",167,0.093926,152,15,0.089820,0.422617,0.020231,0.205229,inf,False


In [220]:
var = vars_top[121]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PERCENT_GAR_BCP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PERCENT_GAR_BCP,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.202285,missing,True
1,PERCENT_GAR_BCP,"[-inf,0.1)",891,0.501125,830,61,0.068462,0.127896,0.008672,0.202285,0.1,False
2,PERCENT_GAR_BCP,"[0.1,0.6000000000000001)",568,0.319460,538,30,0.052817,-0.148214,0.006577,0.202285,0.6000000000000001,False
3,PERCENT_GAR_BCP,"[0.6000000000000001,0.9)",90,0.050619,78,12,0.133333,0.866646,0.055816,0.202285,0.9,False
4,PERCENT_GAR_BCP,"[0.9,inf)",112,0.062992,111,1,0.008929,-1.971083,0.112761,0.202285,inf,False


In [221]:
var = vars_top[122]
print(">>>>> ", var)
display(cortes[var])

>>>>>  CASH_OUT_TOTAL_MES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,CASH_OUT_TOTAL_MES,missing,103,0.057930,97,6,0.058252,-0.044504,0.000113,0.201701,missing,True
1,CASH_OUT_TOTAL_MES,"[-inf,200000.0)",710,0.399325,672,38,0.053521,-0.134225,0.006784,0.201701,200000.0,False
2,CASH_OUT_TOTAL_MES,"[200000.0,600000.0)",242,0.136108,227,15,0.061983,0.021548,0.000064,0.201701,600000.0,False
3,CASH_OUT_TOTAL_MES,"[600000.0,1200000.0)",180,0.101237,175,5,0.027778,-0.816900,0.047784,0.201701,1200000.0,False
4,CASH_OUT_TOTAL_MES,"[1200000.0,2000000.0)",140,0.078740,121,19,0.135714,0.887096,0.091789,0.201701,2000000.0,False
5,CASH_OUT_TOTAL_MES,"[2000000.0,3600000.0)",159,0.089426,150,9,0.056604,-0.074963,0.000486,0.201701,3600000.0,False
6,CASH_OUT_TOTAL_MES,"[3600000.0,6400000.0)",99,0.055681,88,11,0.111111,0.659006,0.032395,0.201701,6400000.0,False
7,CASH_OUT_TOTAL_MES,"[6400000.0,inf)",145,0.081552,140,5,0.034483,-0.593757,0.022287,0.201701,inf,False


In [222]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M', cortes=[1, 2], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3401869158878505  PRECISION:  0.07172413793103448  F1:  0.11847030105777054  GINI:  0.07199836983297181
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_BANCOS_NOIBK_DEBT_SUBIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.2457943925233645  PRECISION:  0.0763203714451538  F1:  0.11647475642161205  GINI:  0.06459206985457255
//////////////////////////////////////////////////


In [223]:
var = vars_top[123]
print(">>>>> ", var)
display(cortes[var])

>>>>>  DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,missing,87,0.048931,87,0,0.000000,-1.841120,0.080699,0.200835,missing,True
1,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,"[-inf,0.1)",99,0.055681,93,6,0.060606,-0.010691,0.000006,0.200835,0.1,False
2,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,"[0.1,0.30000000000000004)",124,0.069741,107,17,0.137097,0.890533,0.081960,0.200835,0.30000000000000004,False
3,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,"[0.30000000000000004,0.8)",674,0.379078,636,38,0.056380,-0.087464,0.002790,0.200835,0.8,False
4,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,"[0.8,0.9)",154,0.086614,144,10,0.064935,0.062921,0.000352,0.200835,0.9,False
5,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,"[0.9,1.0)",317,0.178290,304,13,0.041009,-0.421929,0.026438,0.200835,1.0,False
6,DEBT_PRESTAMOS_VIGENTE_PERCENT_ULT_RCC,"[1.0,inf)",323,0.181665,299,24,0.074303,0.207759,0.008590,0.200835,inf,False


In [224]:
var = vars_top[124]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_IBK_MES_ANTERIOR_y


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_IBK_MES_ANTERIOR_y,missing,334,0.187852,328,6,0.017964,-1.262806,0.177868,0.200616,missing,True
1,MAX_DIAS_ATRASO_IBK_MES_ANTERIOR_y,"[-inf,inf)",1444,0.812148,1342,102,0.070637,0.161504,0.022748,0.200616,inf,False


In [225]:
data = add_new_var_x_cut_up(data, colum='NRO_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M', cortes=[2,3,4,5], impute=0, up=True, soft=False)

>>  _2_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.49439252336448597  PRECISION:  0.06553518334985134  F1:  0.11572959964996717  GINI:  0.06498343763068193
//////////////////////////////////////////////////
>>  _3_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.29345794392523367  PRECISION:  0.07203487038311539  F1:  0.11567507828329342  GINI:  0.06318355021010214
//////////////////////////////////////////////////
>>  _4_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.1766355140186916  PRECISION:  0.06860254083484574  F1:  0.0988235294117647  GINI:  0.03055786401299887
//////////////////////////////////////////////////
>>  _5_cut_up_NUM_BANCOS_NOIBK_DEBT_CAIDA_SEGUIDA_DE_U3M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09065420560747664  PRECISION:  0.07023895727733527

In [226]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GAR_NO_PREFER', cortes=[0.8, 0.9], impute=0, up=True, soft=False)

>>  _0.8_cut_up_PERCENT_GAR_NO_PREFER
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.3691588785046729  PRECISION:  0.048573536645351696  F1:  0.0858509019778309  GINI:  -0.07129426962239083
//////////////////////////////////////////////////
>>  _0.9_cut_up_PERCENT_GAR_NO_PREFER
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.27850467289719627  PRECISION:  0.04456407955735008  F1:  0.07683382751063556  GINI:  -0.08520932004371229
//////////////////////////////////////////////////


In [227]:
var = vars_top[125]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y,missing,334,0.187852,328,6,0.017964,-1.262806,0.177868,0.200616,missing,True
1,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_y,"[-inf,inf)",1444,0.812148,1342,102,0.070637,0.161504,0.022748,0.200616,inf,False


In [228]:
var = vars_top[126]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,missing,334,0.187852,328,6,0.017964,-1.262806,0.177868,0.200616,missing,True
1,MAX_DIAS_ATRASO_IBK_FUENTE_INTERNA_x,"[-inf,inf)",1444,0.812148,1342,102,0.070637,0.161504,0.022748,0.200616,inf,False


In [229]:
var = vars_top[127]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO,missing,334,0.187852,328,6,0.017964,-1.262806,0.177868,0.200616,missing,True
1,MAX_DIAS_ATRASO,"[-inf,inf)",1444,0.812148,1342,102,0.070637,0.161504,0.022748,0.200616,inf,False


In [230]:
data = add_new_var_x_cut_up(data, colum='_int_FM_category_quartil', cortes=[2, 4], impute=0, up=False, soft=True)

>>  _2_cut_down__int_FM_category_quartil
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.09813084112149532  PRECISION:  0.05185185185185185  F1:  0.0678513731825525  GINI:  -0.011171219677775901
//////////////////////////////////////////////////
>>  _4_cut_down__int_FM_category_quartil
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.32897196261682243  PRECISION:  0.056801678231402294  F1:  0.09687629007843676  GINI:  -0.0037731130976258465
//////////////////////////////////////////////////


In [231]:
var = vars_top[128]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MAX_DIAS_ATRASO_IBK_MES_ANTERIOR_x


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MAX_DIAS_ATRASO_IBK_MES_ANTERIOR_x,missing,334,0.187852,328,6,0.017964,-1.262806,0.177868,0.200616,missing,True
1,MAX_DIAS_ATRASO_IBK_MES_ANTERIOR_x,"[-inf,inf)",1444,0.812148,1342,102,0.070637,0.161504,0.022748,0.200616,inf,False


In [232]:
var = vars_top[129]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_CONCENTRACION_PROVEEDORES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_CONCENTRACION_PROVEEDORES,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_CONCENTRACION_PROVEEDORES,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [233]:
var = vars_top[130]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_INVERSION_ACTIVO_PATRIMONIO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_INVERSION_ACTIVO_PATRIMONIO,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_INVERSION_ACTIVO_PATRIMONIO,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [234]:
var = vars_top[131]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_PROCESO_INTEGRACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_PROCESO_INTEGRACION,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_PROCESO_INTEGRACION,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [235]:
var = vars_top[132]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_PRESTAMO_DESVIO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_PRESTAMO_DESVIO,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_PRESTAMO_DESVIO,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [236]:
var = vars_top[133]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_MODELO_NEGOCIO


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_MODELO_NEGOCIO,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_MODELO_NEGOCIO,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [237]:
var = vars_top[134]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_GESTION_FINANCIERA


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_GESTION_FINANCIERA,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_GESTION_FINANCIERA,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [238]:
data = add_new_var_x_cut_up(data, colum='PERCENT_GAR_AUTO_HIPO_PREN', cortes=[0.9, 0.7, 0.5], impute=0, up=False, soft=True)

>>  _0.9_cut_down_PERCENT_GAR_AUTO_HIPO_PREN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.905607476635514  PRECISION:  0.061905066121510255  F1:  0.11588829755426658  GINI:  0.0696744241477536
//////////////////////////////////////////////////
>>  _0.7_cut_down_PERCENT_GAR_AUTO_HIPO_PREN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.8364485981308412  PRECISION:  0.0609050697516162  F1:  0.11354265778623533  GINI:  0.05084003613607857
//////////////////////////////////////////////////
>>  _0.5_cut_down_PERCENT_GAR_AUTO_HIPO_PREN
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.7364485981308411  PRECISION:  0.06015267175572519  F1:  0.11122088920254058  GINI:  0.03554913325551379
//////////////////////////////////////////////////


In [239]:
var = vars_top[135]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_BACKLOG


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_BACKLOG,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_BACKLOG,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [240]:
var = vars_top[136]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_OPERACIONES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_OPERACIONES,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_OPERACIONES,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [241]:
var = vars_top[137]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_ZONA_CLIENTES


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_ZONA_CLIENTES,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_ZONA_CLIENTES,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [242]:
var = vars_top[138]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_CONCENTRACION_VENTAS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_CONCENTRACION_VENTAS,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_CONCENTRACION_VENTAS,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [243]:
var = vars_top[139]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_LINEAS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_LINEAS,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_LINEAS,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [244]:
var = vars_top[139]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_LINEAS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_LINEAS,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_LINEAS,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [245]:
var = vars_top[140]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_GERENCIA_GENERAL


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_GERENCIA_GENERAL,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_GERENCIA_GENERAL,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [246]:
var = vars_top[141]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_MIX_VENTAS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_MIX_VENTAS,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_MIX_VENTAS,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [247]:
var = vars_top[142]
print(">>>>> ", var)
display(cortes[var])

>>>>>  FLG_CAMBIO_ACCIONISTAS


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,FLG_CAMBIO_ACCIONISTAS,missing,1622,0.912261,1515,107,0.065968,0.088106,0.007362,0.199954,missing,True
1,FLG_CAMBIO_ACCIONISTAS,"[-inf,inf)",156,0.087739,155,1,0.006410,-2.304977,0.192593,0.199954,inf,False


In [248]:
var = vars_top[143]
print(">>>>> ", var)
display(cortes[var])

>>>>>  MONTO_ATRASO_TOTAL_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,MONTO_ATRASO_TOTAL_SF,"[-inf,100000.0)",1624,0.913386,1541,83,0.051108,-0.182899,0.028210,0.197485,100000.0,False
1,MONTO_ATRASO_TOTAL_SF,"[100000.0,inf)",154,0.086614,129,25,0.162338,1.097511,0.169276,0.197485,inf,False


In [249]:
var = vars_top[144]
print(">>>>> ", var)
display(cortes[var])

>>>>>  VENTAS_NETAS_ANUAL_CEF_SITUACION


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,VENTAS_NETAS_ANUAL_CEF_SITUACION,missing,169,0.095051,159,10,0.059172,-0.027871,0.000073,0.197431,missing,True
1,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[-inf,5000.0)",103,0.057930,95,8,0.077670,0.264012,0.004538,0.197431,5000.0,False
2,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[5000.0,25000.0)",366,0.205849,353,13,0.035519,-0.563071,0.051243,0.197431,25000.0,False
3,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[25000.0,40000.0)",230,0.129359,218,12,0.052174,-0.161141,0.003131,0.197431,40000.0,False
4,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[40000.0,55000.0)",228,0.128234,214,14,0.061404,0.011529,0.000017,0.197431,55000.0,False
5,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[55000.0,75000.0)",179,0.100675,156,23,0.128492,0.824086,0.098519,0.197431,75000.0,False
6,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[75000.0,110000.0)",248,0.139483,239,9,0.036290,-0.540791,0.032329,0.197431,110000.0,False
7,VENTAS_NETAS_ANUAL_CEF_SITUACION,"[110000.0,inf)",255,0.143420,236,19,0.074510,0.219055,0.007581,0.197431,inf,False


In [250]:
var = vars_top[145]
print(">>>>> ", var)
display(cortes[var])

>>>>>  TIENE_MTOSALDO_OTROS_BANCOS_PRIN_MAYOR_0.9_V


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,TIENE_MTOSALDO_OTROS_BANCOS_PRIN_MAYOR_0.9_V,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.196534,missing,True
1,TIENE_MTOSALDO_OTROS_BANCOS_PRIN_MAYOR_0.9_V,"[-inf,1.0)",1096,0.616423,1011,85,0.077555,0.254105,0.044505,0.196534,1.0,False
2,TIENE_MTOSALDO_OTROS_BANCOS_PRIN_MAYOR_0.9_V,"[1.0,inf)",588,0.330709,565,23,0.039116,-0.471183,0.059897,0.196534,inf,False


In [251]:
var = vars_top[146]
print(">>>>> ", var)
display(cortes[var])

>>>>>  cat_COVER_GAR_AUTO_HIPO_ULT_RCC


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,cat_COVER_GAR_AUTO_HIPO_ULT_RCC,missing,94,0.052868,94,0,0.000000,-1.918506,0.092132,0.195921,missing,True
1,cat_COVER_GAR_AUTO_HIPO_ULT_RCC,"sobre cubierto en 20%%,%sobre cubierto a mas del 60%%,%sobre cubierto de 20% a 40%",218,0.122610,212,6,0.027523,-0.834678,0.059971,0.195921,"sobre cubierto en 20%%,%sobre cubierto a mas del 60%%,%sobre cubierto de 20% a 40%",False
2,cat_COVER_GAR_AUTO_HIPO_ULT_RCC,menos del 10%,709,0.398763,668,41,0.057828,-0.060567,0.001424,0.195921,menos del 10%,False
3,cat_COVER_GAR_AUTO_HIPO_ULT_RCC,"entre10 y 20%%%,%sobre cubierto de 40% a 60%",757,0.425759,696,61,0.080581,0.295673,0.042394,0.195921,"entre10 y 20%%%,%sobre cubierto de 40% a 60%",False


In [252]:
var = vars_top[147]
print(">>>>> ", var)
display(cortes[var])

>>>>>  GAR_TOT_BCP


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,GAR_TOT_BCP,missing,117,0.065804,113,4,0.034188,-0.602646,0.018458,0.195902,missing,True
1,GAR_TOT_BCP,"[-inf,500000.0)",790,0.444319,743,47,0.059494,-0.022101,0.000215,0.195902,500000.0,False
2,GAR_TOT_BCP,"[500000.0,2000000.0)",98,0.055118,96,2,0.020408,-1.132753,0.044139,0.195902,2000000.0,False
3,GAR_TOT_BCP,"[2000000.0,5000000.0)",186,0.104612,179,7,0.037634,-0.503028,0.021314,0.195902,5000000.0,False
4,GAR_TOT_BCP,"[5000000.0,7000000.0)",117,0.065804,110,7,0.059829,-0.016123,0.000017,0.195902,7000000.0,False
5,GAR_TOT_BCP,"[7000000.0,10000000.0)",100,0.056243,88,12,0.120000,0.746018,0.043580,0.195902,10000000.0,False
6,GAR_TOT_BCP,"[10000000.0,16500000.0)",99,0.055681,96,3,0.030303,-0.727288,0.021606,0.195902,16500000.0,False
7,GAR_TOT_BCP,"[16500000.0,inf)",271,0.152418,245,26,0.095941,0.495286,0.046574,0.195902,inf,False


In [253]:
var = vars_top[148]
print(">>>>> ", var)
display(cortes[var])

>>>>>  PROM_CANT_TRX_U12M_CASHIN


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,PROM_CANT_TRX_U12M_CASHIN,missing,91,0.051181,88,3,0.032967,-0.640277,0.015954,0.19538,missing,True
1,PROM_CANT_TRX_U12M_CASHIN,"[-inf,4.0)",469,0.263780,453,16,0.034115,-0.604856,0.074463,0.19538,4.0,False
2,PROM_CANT_TRX_U12M_CASHIN,"[4.0,6.0)",166,0.093363,152,14,0.084337,0.353624,0.013654,0.19538,6.0,False
3,PROM_CANT_TRX_U12M_CASHIN,"[6.0,30.0)",665,0.374016,626,39,0.058647,-0.037341,0.000513,0.19538,30.0,False
4,PROM_CANT_TRX_U12M_CASHIN,"[30.0,72.0)",220,0.123735,194,26,0.118182,0.728686,0.090775,0.19538,72.0,False
5,PROM_CANT_TRX_U12M_CASHIN,"[72.0,inf)",167,0.093926,157,10,0.059880,-0.015213,0.000022,0.19538,inf,False


In [254]:
var = vars_top[149]
print(">>>>> ", var)
display(cortes[var])

>>>>>  VARACION_NUM_BANCOS_FULL_RCCC_SF


,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,VARACION_NUM_BANCOS_FULL_RCCC_SF,missing,1,0.000562,1,0,0.000000,2.624788,0.020121,0.194025,missing,True
1,VARACION_NUM_BANCOS_FULL_RCCC_SF,"[-inf,0.0)",272,0.152981,244,28,0.102941,0.565185,0.062741,0.194025,0.0,False
2,VARACION_NUM_BANCOS_FULL_RCCC_SF,"[0.0,1.0)",1261,0.709224,1186,75,0.059477,-0.030705,0.000659,0.194025,1.0,False
3,VARACION_NUM_BANCOS_FULL_RCCC_SF,"[1.0,inf)",244,0.137233,239,5,0.020492,-1.136877,0.110505,0.194025,inf,False


In [255]:
display(cortes['NRO_CLASIF_NO_NORMAL_GRUPO_EECON'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_CLASIF_NO_NORMAL_GRUPO_EECON,missing,49,0.027559,46,3,0.061224,0.008419,0.000002,0.004609,missing,True
1,NRO_CLASIF_NO_NORMAL_GRUPO_EECON,"[-inf,1.0)",1640,0.922385,1539,101,0.061585,0.014680,0.000200,0.004609,1.0,False
2,NRO_CLASIF_NO_NORMAL_GRUPO_EECON,"[1.0,inf)",89,0.050056,85,4,0.044944,-0.317909,0.004407,0.004609,inf,False


In [256]:
data = add_new_var_x_cut_up(data, colum='NRO_CLASIF_NO_NORMAL_GRUPO_EECON', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_CLASIF_NO_NORMAL_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.05700934579439252  PRECISION:  0.08603667136812412  F1:  0.06857785272625071  GINI:  0.020119900274638614
//////////////////////////////////////////////////


In [257]:
display(cortes['NRO_FEVE_DURO_GRUPO_EECON'])

,variable,bin,count,count_distr,good,bad,badprob,woe,bin_iv,total_iv,breaks,is_special_values
0,NRO_FEVE_DURO_GRUPO_EECON,missing,49,0.027559,46,3,0.061224,0.008419,1.960412e-06,0.000002,missing,True
1,NRO_FEVE_DURO_GRUPO_EECON,"[-inf,inf)",1729,0.972441,1624,105,0.060729,-0.000239,5.576999e-08,0.000002,inf,False


In [258]:
data = add_new_var_x_cut_up(data, colum='NRO_FEVE_DURO_GRUPO_EECON', cortes=[1], impute=0, up=True, soft=False)

>>  _1_cut_up_NUM_FEVE_DURO_GRUPO_EECON
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.016822429906542057  PRECISION:  0.09183673469387756  F1:  0.028436018957345974  GINI:  0.006689218019942933
//////////////////////////////////////////////////


In [259]:
data = add_new_var_x_cut_up(data, colum='BUCKET', cortes=[2,3,4], impute=0, up=True, soft=False)

>>  _2_cut_up_BUCKET
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.4392523364485981  PRECISION:  0.0702646135446255  F1:  0.12114963268462431  GINI:  0.08521613014095841
//////////////////////////////////////////////////
>>  _3_cut_up_BUCKET
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.21495327102803738  PRECISION:  0.07271577616187164  F1:  0.10866997401370185  GINI:  0.04798298752581709
//////////////////////////////////////////////////
>>  _4_cut_up_BUCKET
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.18130841121495328  PRECISION:  0.07695358984529949  F1:  0.10804789752158174  GINI:  0.04883659065250301
//////////////////////////////////////////////////


## OTRAS VARIABLES

In [260]:
data = add_new_var_x_cut_up(data, colum='TENDENCIA_PROTESTOS_U7M', cortes=[0], impute=0, up=True, soft=True)
data = add_new_var_x_cut_up(data, colum='NRO_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS', cortes=[2], impute=0, up=True, soft=False)

>>  _0_cut_up_TENDENCIA_PROTESTOS_U7M
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.03457943925233645  PRECISION:  0.06788990825688074  F1:  0.045820433436532505  GINI:  0.005659935665862648
//////////////////////////////////////////////////
>>  _2_cut_up_NUM_MESES_ANTERIORES_CON_PROTESTOS_CONSECUTIVOS
Se imputará con:  0
Metricas::::::::::::::::::::::   RECALL:  0.04579439252336449  PRECISION:  0.0831918505942275  F1:  0.0590717299578059  GINI:  0.015053187923569578
//////////////////////////////////////////////////


In [261]:
#for var in [col for col in data.columns if 'protesto' in col.lower()]:
#    try:
#        print(">"*20, ' ', var)
#        display(cortes[var])
#    except Exception as e:
#        print(str(e))
        

In [262]:
[col for col in data.columns if 'DESCUENTOS' in col.upper() and 'PER'  in col.upper()]

['DEBT_DESCUENTOS_VENCIDO_PERCENT_ULT_RCC',
 'DEBT_DESCUENTOS_VIGENTE_PERCENT_ULT_RCC']

In [263]:
data['TIENE_COMEX_MAYOR_10_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.10, axis=1).astype(int)
data['TIENE_COMEX_MAYOR_15_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.15, axis=1).astype(int)
data['TIENE_COMEX_MAYOR_20_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.20, axis=1).astype(int)
data['TIENE_COMEX_MAYOR_25_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.25, axis=1).astype(int)
data['TIENE_COMEX_MAYOR_29_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.29, axis=1).astype(int)
data['TIENE_COMEX_MAYOR_30_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.30, axis=1).astype(int)
data['TIENE_COMEX_MAYOR_35_PERCENT_DEUDA_SF'] = data[['DEBT_COMEX_VIGENTE_PERCENT_ULT_RCC', 'DEBT_COMEX_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.35, axis=1).astype(int)

In [264]:
data['TIENE_SOBREGIR_MAYOR_10_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.10, axis=1).astype(int)
data['TIENE_SOBREGIRO_MAYOR_15_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.15, axis=1).astype(int)
data['TIENE_SOBREGIRO_MAYOR_20_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.20, axis=1).astype(int)
data['TIENE_SOBREGIRO_MAYOR_25_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.25, axis=1).astype(int)
data['TIENE_SOBREGIRO_MAYOR_29_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.29, axis=1).astype(int)
data['TIENE_SOBREGIRO_MAYOR_30_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.30, axis=1).astype(int)
data['TIENE_SOBREGIRO_MAYOR_35_PERCENT_DEUDA_SF'] = data[['DEBT_SOBREGIROS__AVANCES_VIGENTE_PERCENT_ULT_RCC', 'DEBT_SOBREGIROS__AVANCES_VENCIDO_PERCENT_ULT_RCC']].apply(lambda _: sum(_) >= 0.35, axis=1).astype(int)

### IMPUTANDO VARIABLES DICOTOMIAS

In [265]:
for col in [_ for _ in data.columns if 'TIENE' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CUT' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MENOR' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'MAYOR' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'SUBIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NUM' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'INT' in _.upper()]:
    data[col] = data[col].fillna(0)

for col in [_ for _ in data.columns if 'CAIDA' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'NRO' in _.upper()]:
    data[col] = data[col].fillna(0)
    
for col in [_ for _ in data.columns if 'COVID' in _.upper()]:
    data[col] = data[col].fillna(0)

### SAVE

In [266]:
data.columns = [col.replace(' ', '_') for col in data.columns]

In [267]:
len([col for col in data.columns if 'and' not in col])

2788

In [286]:
data.shape

(24014, 2824)

In [287]:
[col for col in data.columns if 'fusion'in col]

['target_fusion_atraso_15_feve_all',
 'target_fusion_atraso_15_feve_all_2Mbueno',
 'target_fusion_atraso_feve',
 'target_FINAL_fusion_atraso_feve_2Mbueno',
 'target_FINAL_fusion_atraso_SEGUIDO_15000_FEVE_2Mbueno',
 'target_fusion_atraso_feve_3_3Mbueno',
 'target_fusion_atraso_feve_3_1Mbueno',
 'target_fusion_atraso_feve_3',
 'target_fusion_atraso_feve_2']

In [288]:
data[[col for col in data.columns if 'and' not in col.lower()]].to_csv("AVANCE_6_GRAN_EMPRESA.csv", index=False)